<a href="https://colab.research.google.com/github/matsunagalab/mdzen/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
## Step 3: Visualize Final Structure

View the final frame of the simulation with water molecules wrapped into the periodic box.

---
## Setup: Install Dependencies

**First time only** - This installs AmberTools, OpenMM, and other required packages.

⏱️ Takes ~2-4 minutes. You can continue reading while it runs.

In [ ]:
#@title ▶️ Run Setup (click to expand code)
import sys
import os
import time

IN_COLAB = 'google.colab' in sys.modules

#==============================================================================
# API Key Configuration
#==============================================================================
# os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'
#==============================================================================

def load_dotenv():
    for env_path in ['./.env', '../.env', '/content/.env', '/content/mdzen/.env']:
        try:
            with open(env_path) as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith('#') and '=' in line:
                        key, value = line.split('=', 1)
                        os.environ[key.strip()] = value.strip().strip('"').strip("'")
            return True
        except FileNotFoundError:
            continue
    return False

load_dotenv()

if IN_COLAB:
    try:
        from google.colab import userdata
        for k in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GOOGLE_API_KEY']:
            try:
                v = userdata.get(k)
                if v: os.environ[k] = v
            except: pass
    except: pass

detected = None
for k, p in [('ANTHROPIC_API_KEY', 'anthropic'), ('OPENAI_API_KEY', 'openai'), ('GOOGLE_API_KEY', 'google')]:
    if os.environ.get(k):
        detected = p
        print(f"✓ {k}")
        break
if not detected:
    print("⚠️ No API key found!")

if IN_COLAB:
    start_time = time.time()
    os.chdir('/content')
    
    # Install Miniforge
    print("📦 Installing Miniforge...")
    !curl -fsSL https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -o /tmp/miniforge.sh
    !bash /tmp/miniforge.sh -b -p /usr/local -u 2>&1 | tail -1
    os.environ["PATH"] = f"/usr/local/bin:{os.environ['PATH']}"
    print(f"✓ Miniforge ({time.time() - start_time:.0f}s)")

    # Install scientific packages to base conda
    print("⚗️ Installing scientific packages...")
    !mamba install -y -q openmm pdbfixer parmed ambertools rdkit 2>&1 | tail -2
    print(f"✓ Conda packages ({time.time() - start_time:.0f}s)")

    # Clone repository
    print("📥 Cloning repository...")
    !rm -rf /content/mdzen
    !git clone -q https://github.com/matsunagalab/mdzen.git /content/mdzen
    os.chdir('/content/mdzen')

    # Install uv then pip packages to BOTH Pythons
    print("📦 Installing Python packages (uv)...")
    !pip install -q uv
    
    # Install to conda Python (for MCP servers via stdio)
    CONDA_PYTHON = "/usr/local/bin/python"
    !uv pip install --python {CONDA_PYTHON} -q \
        "litellm>=1.60.0,<1.80.0" \
        anthropic google-genai google-adk \
        "fastmcp>=2.0.0" "mcp[cli]" \
        gradio py3Dmol nest_asyncio \
        mdtraj gemmi pdb2pqr propka dimorphite-dl
    
    # Install to system Python (for Jupyter kernel)
    !uv pip install --python {sys.executable} -q \
        py3Dmol nest_asyncio mdtraj \
        google-adk litellm anthropic \
        "fastmcp>=2.0.0" "mcp[cli]"
    print(f"✓ Pip packages ({time.time() - start_time:.0f}s)")

    # Environment setup
    os.environ["AMBERHOME"] = "/usr/local"
    sys.path.insert(0, '/content/mdzen/src')
    sys.path.insert(0, '/content/mdzen')

    # Verify imports
    try:
        import fastmcp
        from google.adk.runners import Runner
        print("✓ Core packages verified")
    except ImportError as e:
        print(f"⚠️ Import: {e}")

    # Start MCP HTTP servers
    print("🚀 Starting MCP servers...")
    import subprocess
    servers = [
        ("research_server.py", 8001),
        ("structure_server.py", 8002),
        ("genesis_server.py", 8003),
        ("solvation_server.py", 8004),
        ("amber_server.py", 8005),
        ("md_simulation_server.py", 8006),
    ]
    for server, port in servers:
        proc = subprocess.Popen(
            ["/usr/local/bin/python", f"/content/mdzen/servers/{server}", "--http", "--port", str(port)],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL,
            env={**os.environ, "PYTHONPATH": "/content/mdzen/src"}
        )
        print(f"   ✓ {server} (:{port})")
    
    # Wait for servers to start
    import time
    time.sleep(5)
    print("✓ MCP servers started (Streamable HTTP)")

    # Suppress async cleanup warnings
    import logging
    class F(logging.Filter):
        def filter(self, r): return 'cancel scope' not in r.getMessage()
    logging.getLogger('asyncio').addFilter(F())

    print(f"\n✅ Setup complete! ({(time.time() - start_time)/60:.1f} min)")
else:
    sys.path.insert(0, './src')
    load_dotenv()
    print("Local environment")


✓ ANTHROPIC_API_KEY
📦 Installing Miniforge...
ln: failed to create symbolic link '/usr/local/_conda': File exists
✓ Miniforge (4s)
⚗️ Installing scientific packages...
✓ Conda packages (39s)
📥 Cloning repository...
📦 Installing Python packages (uv)...
✓ Pip packages (55s)
🚀 Starting MCP servers...
   ✓ research_server.py (:8001)
   ✓ structure_server.py (:8002)
   ✓ genesis_server.py (:8003)
   ✓ solvation_server.py (:8004)
   ✓ amber_server.py (:8005)
   ✓ md_simulation_server.py (:8006)
✓ 6/6 MCP servers running

✅ Setup complete! (1.1 min)


---
## Step 1: Describe Your Simulation

Tell the AI what you want to simulate in plain language. The AI will ask clarifying questions to help set up the perfect simulation.

In [3]:
#@title 🧬 Step 1a: Describe Your Simulation { display-mode: "form" }
#@markdown ### What do you want to simulate?
user_request = "I want to run MD simulation of PDB 1AKE (adenylate kinase) in water at 300K for 0.1 ns" #@param {type:"string"}

#@markdown ---
#@markdown ### Examples (copy one if you like):
#@markdown - `Setup MD for PDB 1AKE in explicit water, 1 ns at 300K`
#@markdown - `Simulate lysozyme (1LYZ) with TIP3P water model`
#@markdown - `Run equilibrium simulation of ubiquitin (1UBQ) at 310K`
#@markdown - `Setup protein-ligand complex from 3HTB for drug binding study`

import sys
import json
import random
import string
from pathlib import Path

# Initialize session
IN_COLAB = 'google.colab' in sys.modules
if 'mdzen_state' not in dir():
    mdzen_state = {
        "session_id": None, 
        "session_dir": None, 
        "user_request": None,
        "clarification_questions": None,
        "user_answers": None,
        "simulation_brief": None, 
        "workflow_outputs": {}
    }

def init_session():
    job_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    base_dir = Path("/content/mdzen/outputs") if IN_COLAB else Path("./outputs")
    session_dir = base_dir / f"job_{job_id}"
    session_dir.mkdir(parents=True, exist_ok=True)
    mdzen_state["session_id"] = f"job_{job_id}"
    mdzen_state["session_dir"] = str(session_dir)
    return session_dir

if mdzen_state["session_dir"] is None:
    init_session()

# Save user request
mdzen_state["user_request"] = user_request.strip()

print("=" * 60)
print("  ✅ Request Received!")
print("=" * 60)
print(f"  📝 \"{user_request}\"")
print("=" * 60)
print(f"\n📁 Session: {mdzen_state['session_id']}")
print("\n👉 Run the next cell to get AI clarification questions")

  ✅ Request Received!
  📝 "I want to run MD simulation of PDB 1AKE (adenylate kinase) in water at 300K for 0.1 ns"

📁 Session: job_s6iwb47l

👉 Run the next cell to get AI clarification questions


In [4]:
#@title 🤖 Step 1b: Structure Analysis & Clarification { display-mode: "form" }
#@markdown Analyzes structure and generates clarification questions.

import json
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("user_request"):
    print("❌ Error: Please run Step 1a first")
else:
    user_request = mdzen_state["user_request"]
    session_dir = mdzen_state["session_dir"]
    
    print("🤖 Starting clarification agent (Streamable HTTP)...")
    print("-" * 60)
    
    from mdzen.agents.clarification_agent import create_clarification_agent
    from mdzen.tools.mcp_setup import close_toolsets
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    # Use Streamable HTTP transport (more reliable in Colab)
    agent, mcp_tools = create_clarification_agent(transport="http")
    
    session_service = InMemorySessionService()
    runner = Runner(
        app_name="mdzen",
        agent=agent,
        session_service=session_service,
    )
    
    async def run_clarification():
        session = await session_service.create_session(
            app_name="mdzen",
            user_id="colab_user",
            state={"session_dir": session_dir},
        )
        
        message = types.Content(
            role="user",
            parts=[types.Part(text=user_request)],
        )
        
        final_response = None
        async for event in runner.run_async(
            user_id="colab_user",
            session_id=session.id,
            new_message=message,
        ):
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        updated_session = await session_service.get_session(
            app_name="mdzen",
            user_id="colab_user",
            session_id=session.id,
        )
        
        return final_response, updated_session.state
    
    try:
        final_response, session_state = await run_clarification()
        await close_toolsets(mcp_tools)
        
        if session_state.get("simulation_brief"):
            brief = session_state["simulation_brief"]
            if isinstance(brief, str):
                try:
                    brief = json.loads(brief)
                except:
                    pass
            
            if isinstance(brief, dict):
                mdzen_state["simulation_brief"] = brief
                print("\n✅ SimulationBrief Generated!")
                print("-" * 60)
                for key, val in brief.items():
                    if val is not None:
                        print(f"   • {key}: {val}")
                print("\n👉 Proceed to Step 1d to review")
            else:
                mdzen_state["agent_questions"] = brief
                print("\n🤖 Agent needs more information:")
                print("-" * 60)
                print(brief)
                print("\n👉 Answer in Step 1c")
        else:
            if final_response:
                mdzen_state["agent_questions"] = final_response
            print("\n🤖 Agent response:")
            print(final_response or "No response")
            print("\n👉 Answer in Step 1c")
            
    except Exception as e:
        import traceback
        print(f"❌ Error: {e}")
        traceback.print_exc()
        try:
            await close_toolsets(mcp_tools)
        except:
            pass


🤖 Starting clarification agent (ADK Runner + MCP Streamable HTTP)...
------------------------------------------------------------


/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(



🤖 Agent response:
Excellent! Now I have all the information I need. Let me summarize what I found and ask a few clarification questions:

## Structure Analysis: PDB 1AKE

**What I found:**
- **Protein**: Adenylate kinase from *E. coli* (UniProt P69441)
- **Biological unit**: Single chain (214 residues), but the crystal structure contains **2 identical chains** (A and B) - this is likely crystal packing
- **Ligand**: AP5A (bis-adenosine pentaphosphate) inhibitor - present in both chains
- **Crystal waters**: 241 waters in chain A, 137 in chain B

You've already specified some parameters (300K, 0.1 ns), which is great for a quick test run! But I need clarification on a few points:

---

**Question a: Which protein chain to simulate?**
  1. Chain A only (single monomer) - (Recommended)
  2. Both chains A and B (crystal packing dimer)
  3. Other (please specify)

**Question b: What to do with the AP5A ligand?**
  1. Remove it (simulate apo enzyme)
  2. Keep it and parameterize with GAFF2/

In [6]:
#@title 💬 Step 1c: Conversation with Agent { display-mode: "form" }
#@markdown ### Your Response
user_response = "" #@param {type:"string"}
#@markdown Re-run this cell as many times as needed until SimulationBrief is generated.

import json

if 'mdzen_state' not in dir():
    print("❌ Error: Please run Step 1a first")
elif not user_response.strip():
    if mdzen_state.get("simulation_brief") and isinstance(mdzen_state["simulation_brief"], dict):
        print("✅ SimulationBrief already generated! Proceed to Step 1d.")
        brief = mdzen_state["simulation_brief"]
        print(f"   • PDB: {brief.get('pdb_id')} | Chains: {brief.get('select_chains')}")
    elif mdzen_state.get("agent_questions"):
        print("🤖 Agent's questions:")
        print("-" * 50)
        print(mdzen_state["agent_questions"])
        print("-" * 50)
        print("\n👆 Enter your response above and re-run")
    else:
        print("⚠️ Run Step 1b first")
else:
    print(f"💬 Your response: {user_response}")
    print("-" * 50)
    
    from mdzen.agents.clarification_agent import create_clarification_agent
    from mdzen.tools.mcp_setup import close_toolsets
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    session_dir = mdzen_state["session_dir"]
    original_request = mdzen_state.get("user_request", "")
    previous_context = mdzen_state.get("agent_questions", "")
    
    context_message = f"""Original request: {original_request}

Your previous analysis: {previous_context}

User's response: {user_response}

Based on this, either ask follow-up questions OR call generate_simulation_brief with appropriate parameters.
CRITICAL: You must ACTUALLY CALL the tool, not just say you did."""
    
    # Use STDIO transport
    agent, mcp_tools = create_clarification_agent(transport="http")
    session_service = InMemorySessionService()
    runner = Runner(app_name="mdzen", agent=agent, session_service=session_service)
    
    async def run_conversation():
        session = await session_service.create_session(
            app_name="mdzen", user_id="colab_user",
            state={"session_dir": session_dir},
        )
        message = types.Content(role="user", parts=[types.Part(text=context_message)])
        
        print("🔄 Agent thinking...")
        final_response = None
        async for event in runner.run_async(user_id="colab_user", session_id=session.id, new_message=message):
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        updated = await session_service.get_session(app_name="mdzen", user_id="colab_user", session_id=session.id)
        return final_response, updated.state
    
    try:
        final_response, session_state = await run_conversation()
        await close_toolsets(mcp_tools)
        
        if session_state.get("simulation_brief"):
            brief = session_state["simulation_brief"]
            if isinstance(brief, str):
                try: brief = json.loads(brief)
                except: pass
            
            if isinstance(brief, dict):
                mdzen_state["simulation_brief"] = brief
                print("\n" + "=" * 50)
                print("✅ SimulationBrief Generated!")
                print("=" * 50)
                print(f"   • PDB: {brief.get('pdb_id')} | Chains: {brief.get('select_chains')}")
                print(f"   • Temp: {brief.get('temperature')}K | Time: {brief.get('simulation_time_ns')}ns")
                print("\n👉 Proceed to Step 1d")
            else:
                mdzen_state["agent_questions"] = str(brief)
                print("\n🤖 Agent response:")
                print(brief)
        else:
            mdzen_state["agent_questions"] = final_response
            print("\n🤖 Agent response:")
            print(final_response or "No response")
            print("\n👆 Enter response above and re-run")
            
    except Exception as e:
        import traceback
        print(f"❌ Error: {e}")
        traceback.print_exc()


💬 Your response: c 1, d 1
--------------------------------------------------
🔄 Agent is thinking...


/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(



[Debug] Session state keys: ['session_dir', 'simulation_brief']
[Debug] simulation_brief type: <class 'dict'>

✅ SimulationBrief Generated!

   • PDB ID: 1AKE
   • Chains: ['A']
   • Temperature: 300.0 K
   • Duration: 0.1 ns
   • Force Field: ff19SB

👉 Proceed to Step 1d to review the full configuration


In [7]:
#@title ✅ Step 1d: Review & Modify SimulationBrief { display-mode: "form" }
#@markdown ### Current SimulationBrief
#@markdown Run this cell to see the current configuration.
#@markdown 
#@markdown ---
#@markdown ### Modifications (optional)
#@markdown Describe any changes you want (leave empty to keep current):
modifications = "" #@param {type:"string"}
#@markdown 
#@markdown **Examples:**
#@markdown - `Change temperature to 310K`
#@markdown - `Use 0.5 ns simulation time`
#@markdown - `Remove pressure (NVT ensemble)`

import json

if 'mdzen_state' not in dir():
    print("❌ Error: Please run Step 1a first")
elif not mdzen_state.get("simulation_brief"):
    print("❌ Error: No SimulationBrief found")
    print("   Please run Step 1b and 1c first to generate the brief.")
else:
    brief = mdzen_state["simulation_brief"]
    
    # Display current brief
    print("📋 Current SimulationBrief:")
    print("=" * 50)
    
    # Group parameters by category
    structure_keys = ['pdb_id', 'fasta_sequence', 'select_chains', 'structure_file']
    ligand_keys = ['ligand_smiles', 'charge_method', 'atom_type']
    solvation_keys = ['water_model', 'box_padding', 'salt_concentration', 'cubic_box', 
                      'cation_type', 'anion_type', 'is_membrane', 'lipids', 'lipid_ratio']
    simulation_keys = ['temperature', 'pressure_bar', 'simulation_time_ns', 'timestep',
                       'minimize_steps', 'nonbonded_cutoff', 'constraints', 'output_frequency_ps']
    forcefield_keys = ['force_field', 'ph', 'cap_termini', 'include_types']
    
    def print_section(title, keys):
        print(f"\n{title}:")
        for key in keys:
            if key in brief and brief[key] is not None:
                val = brief[key]
                if isinstance(val, list):
                    val = ", ".join(str(v) for v in val)
                elif isinstance(val, dict):
                    val = json.dumps(val)
                print(f"  • {key}: {val}")
    
    print_section("📦 Structure", structure_keys)
    print_section("💊 Ligand", ligand_keys)
    print_section("💧 Solvation", solvation_keys)
    print_section("🌡️ Simulation", simulation_keys)
    print_section("⚗️ Force Field", forcefield_keys)
    
    print("\n" + "=" * 50)
    
    # Handle modifications
    if modifications.strip():
        print(f"\n🔄 Applying modifications: {modifications}")
        print("-" * 50)
        
        # Import necessary modules
        from mdzen.agents.clarification_agent import create_clarification_agent
        from mdzen.tools.mcp_setup import close_toolsets
        from google.adk.runners import Runner
        from google.adk.sessions import InMemorySessionService
        from google.genai import types
        
        session_dir = mdzen_state["session_dir"]
        
        # Create agent
        agent, mcp_tools = create_clarification_agent(transport="http")
        session_service = InMemorySessionService()
        runner = Runner(
            app_name="mdzen",
            agent=agent,
            session_service=session_service,
        )
        
        async def apply_modifications():
            session = await session_service.create_session(
                app_name="mdzen",
                user_id="colab_user",
                state={"session_dir": session_dir, "simulation_brief": brief},
            )
            
            # Ask agent to modify the brief
            modify_prompt = f"""The current SimulationBrief is:
{json.dumps(brief, indent=2)}

The user wants to make these modifications:
{modifications}

Please call generate_simulation_brief with the updated parameters.
Keep all other parameters the same unless the user's modification affects them."""
            
            message = types.Content(
                role="user",
                parts=[types.Part(text=modify_prompt)],
            )
            
            final_response = None
            async for event in runner.run_async(
                user_id="colab_user",
                session_id=session.id,
                new_message=message,
            ):
                if event.is_final_response() and event.content:
                    final_response = event.content.parts[0].text if event.content.parts else None
            
            updated_session = await session_service.get_session(
                app_name="mdzen",
                user_id="colab_user",
                session_id=session.id,
            )
            
            return final_response, updated_session.state
        
        try:
            final_response, session_state = await apply_modifications()
            await close_toolsets(mcp_tools)
            
            if session_state.get("simulation_brief"):
                new_brief = session_state["simulation_brief"]
                if isinstance(new_brief, str):
                    try:
                        new_brief = json.loads(new_brief)
                    except:
                        pass
                
                if isinstance(new_brief, dict):
                    mdzen_state["simulation_brief"] = new_brief
                    print("\n✅ SimulationBrief updated!")
                    print("-" * 50)
                    
                    # Show changes
                    for key in new_brief:
                        if key in brief and new_brief[key] != brief[key]:
                            print(f"  ✓ {key}: {brief[key]} → {new_brief[key]}")
                    
                    print("\n👉 Run this cell again to see the full updated brief")
                else:
                    print(f"\n🤖 Agent response: {new_brief[:500] if len(str(new_brief)) > 500 else new_brief}")
            else:
                print(f"\n🤖 Agent response: {final_response[:500] if final_response else 'No response'}")
                
        except Exception as e:
            import traceback
            print(f"\n❌ Error: {e}")
            traceback.print_exc()
            try:
                await close_toolsets(mcp_tools)
            except:
                pass
    else:
        print("\n✅ Ready for Step 2!")
        print("   No modifications requested. Proceed to Step 2 to run the MD workflow.")


📋 Current SimulationBrief:

📦 Structure:
  • pdb_id: 1AKE
  • select_chains: A

💊 Ligand:
  • charge_method: bcc
  • atom_type: gaff2

💧 Solvation:
  • water_model: tip3p
  • box_padding: 12.0
  • salt_concentration: 0.15
  • cubic_box: True
  • cation_type: Na+
  • anion_type: Cl-
  • is_membrane: False

🌡️ Simulation:
  • temperature: 300.0
  • pressure_bar: 1.0
  • simulation_time_ns: 0.1
  • timestep: 2.0
  • minimize_steps: 500
  • nonbonded_cutoff: 10.0
  • constraints: HBonds
  • output_frequency_ps: 10.0

⚗️ Force Field:
  • force_field: ff19SB
  • ph: 7.0
  • cap_termini: False
  • include_types: protein


✅ Ready for Step 2!
   No modifications requested. Proceed to Step 2 to run the MD workflow.


---
## Step 2: Run MD Workflow

This will execute all 4 steps automatically:
1. **prepare_complex** - Download structure and prepare proteins/ligands
2. **solvate** - Add water box and ions  
3. **build_topology** - Generate Amber topology files
4. **run_simulation** - Run MD with OpenMM

Click ▶️ to start. Progress will be shown below.

In [8]:
#@title ⚙️ Step 2: Run Complete Workflow { display-mode: "form" }
#@markdown ### Run Options
run_simulation_step = True #@param {type:"boolean"}
#@markdown > Uncheck to skip the MD simulation (for testing setup only)

import sys
import json
import traceback
from pathlib import Path
import time

if 'mdzen_state' not in dir() or not mdzen_state.get("simulation_brief"):
    print("❌ Error: Please run Step 1 first to configure your simulation")
else:
    brief = mdzen_state["simulation_brief"]
    session_dir = Path(mdzen_state["session_dir"])
    
    print("=" * 60)
    print(f"  🚀 Starting MD Workflow for {brief.get('pdb_id', 'Unknown')}")
    print("  📡 Using Streamable HTTP transport")
    print("=" * 60)
    
    from mdzen.agents.setup_agent import create_setup_agent
    from mdzen.tools.mcp_setup import close_toolsets
    
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    async def run_setup():
        # Create agent with STDIO transport (no HTTP servers needed)
        print("\n🔧 Creating setup agent (HTTP)...")
        agent, mcp_tools = create_setup_agent(transport="http")
        
        session_service = InMemorySessionService()
        runner = Runner(
            app_name="mdzen",
            agent=agent,
            session_service=session_service,
        )
        
        initial_state = {
            "session_dir": str(session_dir),
            "simulation_brief": json.dumps(brief) if isinstance(brief, dict) else brief,
            "completed_steps": json.dumps([]),
            "outputs": json.dumps({}),
        }
        
        session = await session_service.create_session(
            app_name="mdzen",
            user_id="colab_user",
            state=initial_state,
        )
        
        steps_to_run = ["prepare_complex", "solvate", "build_topology"]
        if run_simulation_step:
            steps_to_run.append("run_simulation")
        
        request = f"""Execute the MD setup workflow with the following SimulationBrief:

{json.dumps(brief, indent=2)}

Please run these steps in order: {', '.join(steps_to_run)}

Work in the directory: {session_dir}
"""
        
        message = types.Content(
            role="user",
            parts=[types.Part(text=request)],
        )
        
        print("\n🤖 Setup agent is running...")
        print("   (This may take several minutes)")
        print("-" * 60)
        
        final_response = None
        async for event in runner.run_async(
            user_id="colab_user",
            session_id=session.id,
            new_message=message,
        ):
            if event.content and event.content.parts:
                text = event.content.parts[0].text if hasattr(event.content.parts[0], 'text') else None
                if text and not event.is_final_response():
                    if any(kw in text.lower() for kw in ['step', 'complete', 'running', 'preparing', 'building']):
                        print(f"   {text[:200]}...")
                
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        updated_session = await session_service.get_session(
            app_name="mdzen",
            user_id="colab_user",
            session_id=session.id,
        )
        
        return final_response, updated_session.state, mcp_tools
    
    try:
        start_time = time.time()
        final_response, session_state, mcp_tools = await run_setup()
        await close_toolsets(mcp_tools)
        elapsed = time.time() - start_time
        
        outputs = session_state.get("outputs", {})
        if isinstance(outputs, str):
            try: outputs = json.loads(outputs)
            except: outputs = {}
        
        completed = session_state.get("completed_steps", [])
        if isinstance(completed, str):
            try: completed = json.loads(completed)
            except: completed = []
        
        mdzen_state["workflow_outputs"] = outputs
        
        print()
        print("=" * 60)
        print("  🎉 Workflow Complete!")
        print("=" * 60)
        print(f"  ⏱️ Time: {elapsed/60:.1f} min")
        print(f"  ✅ Steps completed: {', '.join(completed) if completed else 'None'}")
        print(f"  📁 Output: {session_dir}")
        
        if outputs:
            print()
            print("  📦 Generated files:")
            for key, path in outputs.items():
                if path:
                    print(f"     • {key}: {Path(path).name if isinstance(path, str) else path}")
        
        print()
        if final_response:
            print("  📝 Agent summary:")
            summary = final_response[:500] + "..." if len(final_response) > 500 else final_response
            for line in summary.split('\n'):
                print(f"     {line}")
        
        print()
        print("  👉 Run the next cell to visualize the trajectory")
        
    except Exception as e:
        print()
        print("=" * 60)
        print(f"  ❌ Error: {e}")
        print("=" * 60)
        print(traceback.format_exc())


  🚀 Starting MD Workflow for 1AKE
  📡 Using ADK Runner + MCP Streamable HTTP Transport

🔧 Creating setup agent...

🔌 Warming up MCP connections...
   ✅ server_1 connected
   ✅ server_2 connected
   ✅ server_3 connected
   ✅ server_4 connected
   ✅ server_5 connected
   ⚠️ server_6 timeout, retry 2/3...


CancelledError: Cancelled via cancel scope 7b3b7809b110 by <Task pending name='Task-451' coro=<<async_generator_athrow without __name__>()>>

---
## Step 3: Visualize Results

View the trajectory animation with py3Dmol.

In [ ]:
#@title 🔬 Step 3: Visualize Final Structure { display-mode: "form" }
#@markdown ### Visualization Options
style = "cartoon" #@param ["cartoon", "stick", "sphere", "line"]
show_water = True #@param {type:"boolean"}
#@markdown > Show water molecules (wrapped into periodic box)

import py3Dmol
import tempfile
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("workflow_outputs"):
    print("❌ Error: Please run the workflow first (Step 2)")
elif 'trajectory' not in mdzen_state["workflow_outputs"]:
    print("❌ Error: No trajectory found. Make sure 'Run simulation' was checked in Step 2")
else:
    print("📊 Loading final frame...")
    
    import mdtraj as md
    traj = md.load(
        mdzen_state["workflow_outputs"]['trajectory'], 
        top=mdzen_state["workflow_outputs"]['parm7']
    )
    
    # Get final frame
    final_frame = traj[-1]
    
    # Image molecules (wrap into periodic box)
    final_frame.image_molecules(inplace=True)
    
    print(f"   Trajectory: {traj.n_frames} frames, {traj.time[-1]:.1f} ps total")
    print(f"   Showing: Final frame (t = {final_frame.time[0]:.1f} ps)")
    
    # Select atoms to display
    if show_water:
        # All atoms (protein + water + ions)
        display_frame = final_frame
        atom_info = f"{final_frame.n_atoms} atoms (protein + solvent)"
    else:
        # Protein only
        protein_indices = final_frame.topology.select('protein')
        display_frame = final_frame.atom_slice(protein_indices)
        atom_info = f"{display_frame.n_atoms} protein atoms"
    
    print(f"   Atoms: {atom_info}")
    
    # Save to temporary PDB
    with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
        display_frame.save_pdb(tmp.name, force_overwrite=True)
        tmp_path = tmp.name
    
    # Read PDB content
    with open(tmp_path) as f:
        pdb_content = f.read()
    
    # Create viewer
    view = py3Dmol.view(width=800, height=500)
    view.addModel(pdb_content, 'pdb')
    
    # Apply style based on selection
    if show_water:
        # Protein in cartoon, water as small spheres
        if style == "cartoon":
            view.setStyle({'protein': True}, {'cartoon': {'color': 'spectrum'}})
        elif style == "stick":
            view.setStyle({'protein': True}, {'stick': {}})
        elif style == "sphere":
            view.setStyle({'protein': True}, {'sphere': {'radius': 0.5}})
        else:
            view.setStyle({'protein': True}, {'line': {}})
        
        # Water as small blue spheres
        view.setStyle({'resn': 'WAT'}, {'sphere': {'radius': 0.15, 'color': 'lightblue'}})
        view.setStyle({'resn': 'HOH'}, {'sphere': {'radius': 0.15, 'color': 'lightblue'}})
        
        # Ions as spheres
        view.setStyle({'elem': 'Na'}, {'sphere': {'radius': 0.3, 'color': 'purple'}})
        view.setStyle({'elem': 'Cl'}, {'sphere': {'radius': 0.35, 'color': 'green'}})
    else:
        # Protein only
        if style == "cartoon":
            view.setStyle({'cartoon': {'color': 'spectrum'}})
        elif style == "stick":
            view.setStyle({'stick': {}})
        elif style == "sphere":
            view.setStyle({'sphere': {'radius': 0.5}})
        else:
            view.setStyle({'line': {}})
    
    view.zoomTo()
    
    # Cleanup temp file
    Path(tmp_path).unlink()
    
    print()
    print("✅ Final structure displayed")
    print(f"   Style: {style}" + (" + water" if show_water else ""))
    print()
    print("👉 Run the next cell to download all files")
    
    view.show()


---
## Step 4: Download Results

Download all generated files as a ZIP archive.

In [ ]:
#@title 📥 Step 4: Download Results { display-mode: "form" }

import sys
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("session_dir"):
    print("❌ Error: Please run the workflow first")
else:
    session_dir = Path(mdzen_state["session_dir"])
    
    if not session_dir.exists():
        print("❌ Error: Session directory not found")
    else:
        print("=" * 50)
        print(f"  📂 {session_dir.name}")
        print("=" * 50)
        
        files = sorted(session_dir.rglob('*'))
        total_size = 0
        
        for f in files:
            if f.is_file():
                size = f.stat().st_size
                total_size += size
                size_str = f"{size/1024:.1f} KB" if size > 1024 else f"{size} B"
                print(f"  {f.relative_to(session_dir):<40} {size_str:>10}")
        
        print("=" * 50)
        print(f"  Total: {total_size/1024/1024:.2f} MB")
        print("=" * 50)
        
        if 'google.colab' in sys.modules:
            from google.colab import files
            import shutil
            
            zip_path = f"/content/{session_dir.name}.zip"
            shutil.make_archive(zip_path.replace('.zip', ''), 'zip', session_dir)
            
            print(f"\n⬇️ Downloading {session_dir.name}.zip...")
            files.download(zip_path)
        else:
            print(f"\n📁 Files are in: {session_dir}")

---

## What's Next?

🎉 **Congratulations!** You've completed an MD simulation using MDZen.

### Continue Learning
- **Longer simulations**: Increase `simulation_time_ns` in Step 1
- **Different proteins**: Try other PDB IDs like 1LYZ (lysozyme), 1UBQ (ubiquitin)
- **Analysis**: Load trajectory in MDTraj for RMSD, RMSF, hydrogen bonds

### Resources
- [MDZen GitHub](https://github.com/matsunagalab/mdzen)
- [OpenMM Documentation](http://openmm.org/documentation.html)
- [MDTraj Documentation](https://mdtraj.org/)

In [ ]:
# Check if we're in a local environment (not Colab)
import sys
print(f"Python: {sys.version}")
print(f"In Colab: {'google.colab' in sys.modules}")

# Add paths for local development
sys.path.insert(0, './src')
sys.path.insert(0, '.')

# Check if API key is available
import os
from pathlib import Path

# Try to load .env
def load_dotenv():
    for env_path in ['./.env', '../.env']:
        try:
            with open(env_path) as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith('#') and '=' in line:
                        key, value = line.split('=', 1)
                        os.environ[key.strip()] = value.strip().strip('"').strip("'")
                print(f"✓ Loaded .env from {env_path}")
                return True
        except FileNotFoundError:
            continue
    return False

load_dotenv()

# Check API keys
api_key = os.environ.get('ANTHROPIC_API_KEY', '')
if api_key:
    print(f"✓ ANTHROPIC_API_KEY: {api_key[:10]}...")
else:
    print("✗ No ANTHROPIC_API_KEY found")


In [ ]:
# Start MCP servers with Streamable HTTP transport
import subprocess
import sys
import time
import os

# Get the correct path
base_path = os.getcwd()
print(f"Working directory: {base_path}")

MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))
    print(f"   ✓ {server_file} on port {port} (/mcp)")

time.sleep(3)  # Wait for servers to initialize
print(f"\n✓ MCP servers started")


In [ ]:
# Step 1a: Initialize session
import sys
import json
import random
import string
from pathlib import Path

# Initialize session state
IN_COLAB = 'google.colab' in sys.modules
mdzen_state = {
    "session_id": None, 
    "session_dir": None, 
    "user_request": None,
    "clarification_questions": None,
    "user_answers": None,
    "simulation_brief": None, 
    "workflow_outputs": {}
}

def init_session():
    job_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    base_dir = Path("/content/mdzen/outputs") if IN_COLAB else Path("./outputs")
    session_dir = base_dir / f"job_{job_id}"
    session_dir.mkdir(parents=True, exist_ok=True)
    mdzen_state["session_id"] = f"job_{job_id}"
    mdzen_state["session_dir"] = str(session_dir)
    return session_dir

init_session()

# Set user request
user_request = "I want to run MD simulation of PDB 1AKE (adenylate kinase) in water at 300K for 1 ns"
mdzen_state["user_request"] = user_request.strip()

print("=" * 60)
print("  ✅ Request Received!")
print("=" * 60)
print(f"  📝 \"{user_request}\"")
print("=" * 60)
print(f"\n📁 Session: {mdzen_state['session_id']}")
print(f"📂 Directory: {mdzen_state['session_dir']}")


In [ ]:
# Step 1b: Structure Analysis & Clarification (ADK Runner)
import re
import json
from pathlib import Path

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print("🤖 Starting clarification agent (ADK Runner + MCP Streamable HTTP)...")
print("-" * 60)

# Import shared agent (same as main.py!)
from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (Streamable HTTP /mcp endpoint)
agent, mcp_tools = create_clarification_agent(transport="http")

# Create runner (same pattern as main.py!)
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

# Run the agent
async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    # Get simulation brief from session state
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    # Run the async function
    final_response, session_state = await run_clarification()
    
    # Close MCP connections
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        # Parse if JSON string
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("✅ Clarification complete!")
            print("-" * 60)
            print(f"\n📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
            
            print()
            print("=" * 60)
            print("👉 Ready for Step 2")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        # Display agent's response for manual interaction
        print("\n🤖 Agent response:")
        print(final_response or "No response")
        
except Exception as e:
    import traceback
    print(f"❌ Error: {e}")
    print(traceback.format_exc())
    
    # Cleanup
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Reload the modules to pick up changes
import importlib
import sys

# Remove cached modules
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]

print(f"Removed {len(modules_to_remove)} cached mdzen modules")

# Re-import
from mdzen.agents.clarification_agent import create_clarification_agent
import inspect
print(f"create_clarification_agent signature: {inspect.signature(create_clarification_agent)}")


In [ ]:
# Check what's in the content directory
import os
content_file = "/content/mdzen/src/mdzen/agents/clarification_agent.py"
print(f"Reading: {content_file}")

with open(content_file, 'r') as f:
    content = f.read()

# Check if transport parameter exists
if "transport:" in content:
    print("✓ transport parameter found")
else:
    print("✗ transport parameter NOT found - need to update")

# Show function definition line
for i, line in enumerate(content.split('\n'), 1):
    if 'def create_clarification_agent' in line:
        print(f"Line {i}: {line}")
        # Show next few lines
        lines = content.split('\n')
        for j in range(i, min(i+5, len(lines))):
            print(f"Line {j+1}: {lines[j]}")
        break


In [ ]:
# Update clarification_agent.py with transport parameter
clarification_agent_code = '''"""Phase 1: Clarification Agent for MDZen.

This agent handles user interaction to gather MD simulation requirements
and generates a structured SimulationBrief.
"""

from typing import Literal

from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.mcp_tool import McpToolset

from mdzen.config import get_litellm_model
from mdzen.prompts import get_clarification_instruction
from mdzen.tools.mcp_setup import get_clarification_tools, get_clarification_tools_sse
from mdzen.tools.custom_tools import generate_simulation_brief, get_session_dir


def create_clarification_agent(
    transport: Literal["stdio", "sse", "http"] = "stdio",
    sse_host: str = "localhost",
) -> tuple[LlmAgent, list[McpToolset]]:
    """Create the Phase 1 clarification agent.

    This agent:
    1. Gets session_dir via get_session_dir tool
    2. Uses download_structure/inspect_molecules to analyze structures
    3. Asks clarification questions based on inspection
    4. Generates SimulationBrief via generate_simulation_brief tool
    5. Saves result to session.state["simulation_brief"] via output_key

    Args:
        transport: MCP transport mode:
            - "stdio": subprocess-based (default, for CLI)
            - "sse" or "http": HTTP-based using Streamable HTTP /mcp endpoint (for Colab)
        sse_host: Hostname for HTTP servers (only used when transport="sse" or "http")

    Returns:
        Tuple of (LlmAgent, list of McpToolset instances to close after use)
    """
    # Get MCP tools for structure inspection based on transport mode
    if transport in ("sse", "http"):
        mcp_tools = get_clarification_tools_sse(host=sse_host)
    else:
        mcp_tools = get_clarification_tools()

    # Create FunctionTools for session management and brief generation
    get_session_dir_tool = FunctionTool(get_session_dir)
    generate_brief_tool = FunctionTool(generate_simulation_brief)

    # Combine all tools
    all_tools = mcp_tools + [get_session_dir_tool, generate_brief_tool]

    agent = LlmAgent(
        model=LiteLlm(model=get_litellm_model("clarification")),
        name="clarification_agent",
        description="Gathers MD simulation requirements and generates SimulationBrief",
        instruction=get_clarification_instruction(),
        tools=all_tools,
        output_key="simulation_brief",  # Saves to session.state["simulation_brief"]
    )

    return agent, mcp_tools
'''

with open("/content/mdzen/src/mdzen/agents/clarification_agent.py", 'w') as f:
    f.write(clarification_agent_code)

print("✓ Updated clarification_agent.py")


In [ ]:
import subprocess
import os

os.chdir("/content/mdzen")
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print(result.stderr)


In [ ]:
# Kill old MCP server processes and restart with new code
import subprocess
import sys
import os
import time

# Kill existing MCP servers
for server_file, port, proc in mcp_server_procs:
    try:
        proc.terminate()
        proc.wait(timeout=2)
    except:
        proc.kill()

print("✓ Stopped old MCP servers")

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"✓ Cleared {len(modules_to_remove)} cached modules")

# Restart MCP servers
base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))
    print(f"   ✓ {server_file} on port {port}")

time.sleep(3)
print("\n✓ MCP servers restarted with new code")

# Verify transport parameter exists
from mdzen.agents.clarification_agent import create_clarification_agent
import inspect
print(f"\ncreate_clarification_agent signature: {inspect.signature(create_clarification_agent)}")


In [ ]:
# Step 1b: Structure Analysis & Clarification (ADK Runner)
import re
import json
from pathlib import Path

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print("🤖 Starting clarification agent (ADK Runner + MCP Streamable HTTP)...")
print("-" * 60)

# Import shared agent
from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (Streamable HTTP /mcp endpoint)
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created with HTTP transport")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

# Run the agent
async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    # Get simulation brief from session state
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    print("🔄 Running clarification agent...")
    final_response, session_state = await run_clarification()
    
    # Close MCP connections
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("✅ Clarification complete!")
            print("-" * 60)
            print(f"\n📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response or "No response")
        
except Exception as e:
    import traceback
    print(f"❌ Error: {e}")
    print(traceback.format_exc())
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Check if MCP servers are running and listening
import subprocess

# Check running processes
result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
python_procs = [line for line in result.stdout.split('\n') if 'server.py' in line and 'python' in line]
print("Running server processes:")
for proc in python_procs:
    print(f"  {proc[:100]}...")

print()

# Check if ports are listening
result = subprocess.run(["ss", "-tlnp"], capture_output=True, text=True)
print("Listening ports:")
for line in result.stdout.split('\n'):
    if any(str(port) in line for port in [8001, 8002, 8003, 8004, 8005, 8006]):
        print(f"  {line}")

# Try to connect to one of the servers
import socket
for port in [8001, 8002]:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', port))
    if result == 0:
        print(f"✓ Port {port} is open")
    else:
        print(f"✗ Port {port} is NOT open (error: {result})")
    sock.close()


In [ ]:
# Start one server with visible output to see what's happening
import subprocess
import sys
import time

base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print(f"Starting: python {server_path} --http --port 8001")
print("-" * 60)

# Start with captured output
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)

# Wait a bit and check
time.sleep(3)

# Check if process is still running
if proc.poll() is None:
    print("✓ Server process is running")
else:
    print(f"✗ Server exited with code: {proc.returncode}")
    stdout, stderr = proc.communicate()
    if stdout:
        print("STDOUT:", stdout[:1000])
    if stderr:
        print("STDERR:", stderr[:1000])


In [ ]:
# Check if port 8001 is now listening
import socket
import time

time.sleep(2)  # Give more time for server to bind

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ Port 8001 is open")
else:
    print(f"✗ Port 8001 is NOT open (error: {result})")
sock.close()

# Try HTTP request to the /mcp endpoint
import urllib.request
try:
    req = urllib.request.Request('http://localhost:8001/mcp', method='GET')
    with urllib.request.urlopen(req, timeout=5) as response:
        print(f"HTTP response: {response.status}")
except Exception as e:
    print(f"HTTP request result: {e}")


In [ ]:
# Check server process output
import subprocess
import sys

# Kill any existing and start fresh with full logging
subprocess.run(["pkill", "-f", "research_server"], capture_output=True)

import time
time.sleep(1)

base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

# Run server and wait for output
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)

# Wait and read output
time.sleep(5)

# Read any available output
import select
import os

# Make stdout non-blocking
os.set_blocking(proc.stdout.fileno(), False)

output = ""
try:
    output = proc.stdout.read() or ""
except:
    pass

print(f"Process running: {proc.poll() is None}")
print(f"Output so far:\n{output[:2000] if output else '(no output yet)'}")

# Check stderr separately
if proc.poll() is not None:
    print(f"Exit code: {proc.returncode}")


In [ ]:
# Check FastMCP version and run() signature
import fastmcp
import inspect

print(f"FastMCP version: {fastmcp.__version__}")

# Check FastMCP.run signature
from fastmcp import FastMCP
mcp = FastMCP("test")
sig = inspect.signature(mcp.run)
print(f"\nFastMCP.run() signature: {sig}")

# Check available parameters
print("\nrun() parameters:")
for name, param in sig.parameters.items():
    print(f"  {name}: {param.annotation} = {param.default}")


In [ ]:
# Check what transports are available and how to configure them
from fastmcp import FastMCP

# Check if there are transport-specific settings
import fastmcp.server
print(dir(fastmcp.server))

# Try to find how to set port
try:
    from fastmcp.server import serve
    sig = inspect.signature(serve)
    print(f"\nserve() signature: {sig}")
except:
    pass

# Check Transport type
try:
    from fastmcp.server.transports import Transport
    print(f"\nTransport types available:")
    for name in dir(fastmcp.server):
        if 'transport' in name.lower():
            print(f"  {name}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
# Pull latest changes and restart servers
import subprocess
import sys
import os
import time

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill any existing server processes
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"✓ Cleared {len(modules_to_remove)} cached modules")

# Restart servers
base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))

time.sleep(3)

# Check if port 8001 is open
import socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ Port 8001 is open - server is running!")
else:
    print(f"✗ Port 8001 connection failed (error: {result})")
sock.close()


In [ ]:
# Check server startup with visible output
import subprocess
import sys
import time

base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

# Kill any existing
subprocess.run(["pkill", "-f", "research_server"], capture_output=True)
time.sleep(1)

print(f"Starting: {server_path} --http --port 8001")
print("-" * 60)

# Start with captured output
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

# Wait longer for uvicorn to start
time.sleep(5)

# Check if still running
if proc.poll() is None:
    print("✓ Process still running")
    
    # Check port
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    if result == 0:
        print("✓ Port 8001 is listening!")
    else:
        print(f"✗ Port 8001 not open yet (error: {result})")
    sock.close()
else:
    print(f"✗ Process exited with code: {proc.returncode}")
    
# Read any output
import os
os.set_blocking(proc.stdout.fileno(), False)
try:
    output = proc.stdout.read() or ""
    if output:
        print("\nServer output:")
        print(output[:2000])
except:
    pass


In [ ]:
# Check FastMCP run method more carefully
from fastmcp import FastMCP
import inspect

mcp = FastMCP("test")

# Get the source of run method
print("FastMCP.run source:")
print(inspect.getsource(mcp.run)[:2000])


In [ ]:
# Check run_async method
from fastmcp import FastMCP
import inspect

mcp = FastMCP("test")

# Get the source of run_async method
print("FastMCP.run_async source:")
source = inspect.getsource(mcp.run_async)
print(source[:3000])


In [ ]:
# Check run_http_async method
from fastmcp import FastMCP
import inspect

mcp = FastMCP("test")

# Get the signature
sig = inspect.signature(mcp.run_http_async)
print(f"run_http_async signature: {sig}")

# Get the source
print("\nrun_http_async source:")
source = inspect.getsource(mcp.run_http_async)
print(source[:2500])


In [ ]:
# Try setting config via FastMCP settings instead
from fastmcp import FastMCP

# Check if there's a settings/config way
mcp = FastMCP("test")
print(f"FastMCP version: {fastmcp.__version__}")
print(f"Settings attributes: {[a for a in dir(mcp) if 'setting' in a.lower() or 'config' in a.lower()]}")

# Check _deprecated_settings
if hasattr(mcp, '_deprecated_settings'):
    print(f"\n_deprecated_settings: {mcp._deprecated_settings}")
    print(f"host: {mcp._deprecated_settings.host}")
    print(f"port: {mcp._deprecated_settings.port}")


In [ ]:
# Try setting host/port via the deprecated_settings before run
from fastmcp import FastMCP
import fastmcp

print(f"FastMCP version: {fastmcp.__version__}")

# Create server and configure settings
mcp = FastMCP("test")
mcp._deprecated_settings.host = "0.0.0.0"
mcp._deprecated_settings.port = 8099

print(f"Configured host: {mcp._deprecated_settings.host}")
print(f"Configured port: {mcp._deprecated_settings.port}")

# Try run with just transport
# This should work and use the settings we just set
print("\nTrying mcp.run(transport='http') - this should use the settings...")
# Don't actually run it, just verify the approach
print("(Not actually running, just testing approach)")

# Also check if there's a direct way to configure in constructor
print("\nChecking FastMCP constructor signature:")
import inspect
sig = inspect.signature(FastMCP.__init__)
print(f"FastMCP.__init__{sig}")


In [ ]:
# Pull latest and restart servers
import subprocess
import sys
import os
import time
import socket

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill any existing server processes
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]

# Start research_server and test
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print(f"\nStarting research_server on port 8001...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

time.sleep(5)

# Check if running and port open
if proc.poll() is None:
    print("✓ Process is running")
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    if result == 0:
        print("✓ Port 8001 is listening!")
    else:
        print(f"✗ Port 8001 not open (error: {result})")
    sock.close()
else:
    print(f"✗ Process exited with code: {proc.returncode}")
    os.set_blocking(proc.stdout.fileno(), False)
    try:
        output = proc.stdout.read() or ""
        print(f"Output: {output[:1500]}")
    except:
        pass


In [ ]:
# Check FastMCP version in Colab
import fastmcp
print(f"FastMCP version: {fastmcp.__version__}")

from fastmcp import FastMCP
mcp = FastMCP("test")
print(f"Available attributes: {[a for a in dir(mcp) if not a.startswith('__')]}")


In [ ]:
# Run a minimal test to see what's happening
import subprocess
import sys

test_code = '''
import sys
sys.path.insert(0, '/content/mdzen/src')
from fastmcp import FastMCP
import fastmcp
print(f"FastMCP version: {fastmcp.__version__}")
mcp = FastMCP("test")
print(f"Has _deprecated_settings: {hasattr(mcp, '_deprecated_settings')}")
if hasattr(mcp, '_deprecated_settings'):
    print(f"Settings: {mcp._deprecated_settings}")
else:
    print(f"Settings: {mcp.settings if hasattr(mcp, 'settings') else 'No settings'}")
'''

result = subprocess.run([sys.executable, "-c", test_code], capture_output=True, text=True)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr[:500])


In [ ]:
# Check the research_server.py directly
import subprocess
import sys

# Run a test to check what's happening in the server file
result = subprocess.run(
    [sys.executable, "-c", '''
import sys
sys.path.insert(0, "/content/mdzen/src")

# Import what research_server imports
from fastmcp import FastMCP
import fastmcp
print(f"FastMCP version: {fastmcp.__version__}")

# Create mcp same way as research_server
mcp = FastMCP("Research Server")
print(f"mcp type: {type(mcp)}")
print(f"Has _deprecated_settings: {hasattr(mcp, '_deprecated_settings')}")

# Try setting like in the server
try:
    mcp._deprecated_settings.host = "0.0.0.0"
    mcp._deprecated_settings.port = 8001
    print("✓ Settings applied successfully")
except AttributeError as e:
    print(f"✗ Error: {e}")
'''],
    capture_output=True,
    text=True
)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr[:1000])


In [ ]:
# Let's see exactly what's happening in the server file
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "/content/mdzen/servers/research_server.py", "--http", "--port", "8001"],
    capture_output=True,
    text=True,
    timeout=10
)
print("Exit code:", result.returncode)
print("STDOUT:", result.stdout[:500] if result.stdout else "")
print("STDERR:", result.stderr[:1500] if result.stderr else "")


In [ ]:
# Check what's different in the server environment
import subprocess
import sys

# Check if there's a local fastmcp module or something shadowing
result = subprocess.run(
    [sys.executable, "-c", '''
import sys
sys.path.insert(0, "/content/mdzen/src")
sys.path.insert(0, "/content/mdzen/servers")
sys.path.insert(0, "/content/mdzen")

# Debug imports
import fastmcp
print(f"fastmcp location: {fastmcp.__file__}")
print(f"fastmcp version: {fastmcp.__version__}")

from fastmcp import FastMCP
print(f"FastMCP class: {FastMCP}")
print(f"FastMCP module: {FastMCP.__module__}")

# Check all fastmcp-related modules
for name, mod in sorted(sys.modules.items()):
    if 'fastmcp' in name.lower() or 'mcp' in name.lower():
        if hasattr(mod, '__file__'):
            print(f"  {name}: {mod.__file__}")
'''],
    capture_output=True,
    text=True
)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr[:500])


In [ ]:
# Check the first few lines of research_server.py to see how it imports FastMCP
with open("/content/mdzen/servers/research_server.py", "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines[:50], 1):
        if 'import' in line.lower() or 'fastmcp' in line.lower():
            print(f"{i}: {line.rstrip()}")


In [ ]:
# Pull and test
import subprocess
import sys
import os
import time
import socket

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill any existing server processes
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Start research_server and test
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print(f"\nStarting research_server on port 8001...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

time.sleep(5)

# Check if running and port open
if proc.poll() is None:
    print("✓ Process is running")
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    if result == 0:
        print("✓ Port 8001 is listening!")
    else:
        print(f"✗ Port 8001 not open (error: {result})")
    sock.close()
else:
    print(f"✗ Process exited with code: {proc.returncode}")
    os.set_blocking(proc.stdout.fileno(), False)
    try:
        output = proc.stdout.read() or ""
        print(f"Output:\n{output[:2000]}")
    except:
        pass


In [ ]:
# Start all MCP servers
import subprocess
import sys
import os
import time
import socket

# Kill any existing
subprocess.run(["pkill", "-f", "_server.py"], capture_output=True)
time.sleep(1)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]

base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))

time.sleep(5)

# Check all ports
for server_file, port, proc in mcp_server_procs:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', port))
    status = "✓" if result == 0 else "✗"
    print(f"{status} {server_file} on port {port}")
    sock.close()


In [ ]:
# Check what's happening with the servers
import subprocess
import sys
import os
import time

base_path = "/content/mdzen"

# Start one server with output visible
server_path = f"{base_path}/servers/research_server.py"

proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

# Wait and check
time.sleep(8)

if proc.poll() is None:
    print("✓ Server is running")
    
    # Check port
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', 8001))
    print(f"Port check: {'✓ open' if result == 0 else '✗ not open'}")
    sock.close()
else:
    print(f"✗ Server exited with code: {proc.returncode}")
    
# Read output
os.set_blocking(proc.stdout.fileno(), False)
try:
    output = proc.stdout.read() or ""
    if output:
        print(f"\nOutput:\n{output[:1500]}")
except:
    pass


In [ ]:
# Run server directly and capture all output
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "/content/mdzen/servers/research_server.py", "--http", "--port", "8001"],
    capture_output=True,
    text=True,
    timeout=15
)

print("Exit code:", result.returncode)
print("\nSTDOUT:")
print(result.stdout[-3000:] if len(result.stdout) > 3000 else result.stdout)
print("\nSTDERR:")
print(result.stderr[-2000:] if len(result.stderr) > 2000 else result.stderr)


In [ ]:
# Kill all server processes properly
import subprocess
import time
import os

# Kill by port
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    subprocess.run(f"fuser -k {port}/tcp 2>/dev/null", shell=True, capture_output=True)

# Also kill by name
subprocess.run(["pkill", "-9", "-f", "_server.py"], capture_output=True)

time.sleep(2)

# Verify ports are free
import socket
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', port))
    status = "in use" if result == 0 else "free"
    print(f"Port {port}: {status}")
    sock.close()


In [ ]:
# Start all MCP servers with proper waiting
import subprocess
import sys
import time
import socket

base_path = "/content/mdzen"
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

mcp_server_procs = []
for server_file, port in MCP_SERVERS:
    server_path = f"{base_path}/servers/{server_file}"
    proc = subprocess.Popen(
        [sys.executable, server_path, "--http", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    mcp_server_procs.append((server_file, port, proc))
    print(f"Started {server_file}...")
    time.sleep(1)  # Stagger startup

print("\nWaiting for servers to initialize...")
time.sleep(5)

# Check all ports
print("\nServer status:")
all_ok = True
for server_file, port, proc in mcp_server_procs:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', port))
    if result == 0:
        print(f"✓ {server_file} on port {port}")
    else:
        print(f"✗ {server_file} on port {port} (process running: {proc.poll() is None})")
        all_ok = False
    sock.close()

if all_ok:
    print("\n✅ All servers running!")


In [ ]:
# Check structure_server error
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "/content/mdzen/servers/structure_server.py", "--http", "--port", "8002"],
    capture_output=True,
    text=True,
    timeout=10
)

print("Exit code:", result.returncode)
print("\nError output (last 2000 chars):")
print(result.stderr[-2000:] if result.stderr else "(no stderr)")


In [ ]:
# Clear module cache and run clarification agent (only needs research_server)
import sys

# Clear cached mdzen modules
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"Cleared {len(modules_to_remove)} cached modules")

# Now run Step 1b
import json
from pathlib import Path

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print(f"\n🤖 Starting clarification agent...")
print(f"   Request: {user_request}")
print(f"   Session: {session_dir}")
print("-" * 60)

from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    print("🔄 Running agent...")
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    final_response, session_state = await run_clarification()
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("\n✅ Clarification complete!")
            print("-" * 60)
            print("📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:1000] if len(brief) > 1000 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response[:1000] if final_response else "No response")
        
except Exception as e:
    import traceback
    print(f"\n❌ Error: {e}")
    traceback.print_exc()
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Kill existing and restart with SSE mode
import subprocess
import sys
import time
import socket

# Kill all
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    subprocess.run(f"fuser -k {port}/tcp 2>/dev/null", shell=True, capture_output=True)
subprocess.run(["pkill", "-9", "-f", "_server.py"], capture_output=True)
time.sleep(2)

# Start research_server with SSE mode
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print("Starting research_server with SSE mode...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--sse", "--port", "8001"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

time.sleep(5)

# Check port
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ Port 8001 is listening")
else:
    print(f"✗ Port 8001 not open")
sock.close()

# Test SSE endpoint
import urllib.request
try:
    req = urllib.request.Request('http://localhost:8001/sse')
    with urllib.request.urlopen(req, timeout=5) as response:
        print(f"✓ /sse endpoint responded: {response.status}")
except Exception as e:
    print(f"SSE endpoint: {e}")


In [ ]:
# Check what connection types ADK supports
from google.adk.tools.mcp_tool import mcp_session_manager
import inspect

# List all classes in the module
print("Available connection params in ADK:")
for name in dir(mcp_session_manager):
    obj = getattr(mcp_session_manager, name)
    if isinstance(obj, type) and 'Params' in name:
        print(f"  - {name}")
        
# Check SseConnectionParams
from google.adk.tools.mcp_tool.mcp_session_manager import SseConnectionParams
print(f"\nSseConnectionParams fields:")
print(inspect.signature(SseConnectionParams))


In [ ]:
# Check StreamableHTTPConnectionParams signature
from google.adk.tools.mcp_tool.mcp_session_manager import StreamableHTTPConnectionParams
import inspect

print("StreamableHTTPConnectionParams:")
print(inspect.signature(StreamableHTTPConnectionParams))


In [ ]:
# Pull, restart servers, and test
import subprocess
import sys
import os
import time
import socket

os.chdir("/content/mdzen")

# Pull changes
result = subprocess.run(["git", "pull", "origin", "main"], capture_output=True, text=True)
print(result.stdout)

# Kill existing servers
for port in [8001, 8002, 8003, 8004, 8005, 8006]:
    subprocess.run(f"fuser -k {port}/tcp 2>/dev/null", shell=True, capture_output=True)
subprocess.run(["pkill", "-9", "-f", "_server.py"], capture_output=True)
time.sleep(2)

# Clear module cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('mdzen')]
for mod in modules_to_remove:
    del sys.modules[mod]
print(f"Cleared {len(modules_to_remove)} cached modules")

# Start research_server with HTTP mode
base_path = "/content/mdzen"
server_path = f"{base_path}/servers/research_server.py"

print("\nStarting research_server with HTTP mode...")
proc = subprocess.Popen(
    [sys.executable, server_path, "--http", "--port", "8001"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

time.sleep(5)

# Check port
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
result = sock.connect_ex(('localhost', 8001))
if result == 0:
    print("✓ research_server on port 8001")
else:
    print(f"✗ Port 8001 not open")
sock.close()


In [ ]:
# Test clarification agent with Streamable HTTP
import json

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print(f"🤖 Testing clarification agent with Streamable HTTP...")
print(f"   Request: {user_request[:60]}...")
print("-" * 60)

from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (now uses StreamableHTTPConnectionParams)
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created with StreamableHTTPConnectionParams")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    print("🔄 Running agent...")
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    final_response, session_state = await run_clarification()
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("\n✅ SUCCESS! Clarification complete!")
            print("-" * 60)
            print("📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response[:500] if final_response else "No response")
        
except Exception as e:
    import traceback
    print(f"\n❌ Error: {e}")
    traceback.print_exc()
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [ ]:
# Check the generated brief
import json

brief = mdzen_state.get("simulation_brief")
print(f"Brief type: {type(brief)}")
print(f"Brief value:\n{brief}")

# Parse if it's a string
if isinstance(brief, str):
    try:
        brief = json.loads(brief)
        mdzen_state["simulation_brief"] = brief
        print("\n✓ Parsed successfully!")
        print("\n📋 SimulationBrief:")
        for key, val in brief.items():
            if val is not None:
                print(f"   • {key}: {val}")
    except json.JSONDecodeError as e:
        print(f"JSON parse error: {e}")


In [ ]:
# Test clarification agent with Streamable HTTP
import json

user_request = mdzen_state["user_request"]
session_dir = mdzen_state["session_dir"]

print(f"🤖 Testing clarification agent with Streamable HTTP...")
print(f"   Request: {user_request[:60]}...")
print("-" * 60)

from mdzen.agents.clarification_agent import create_clarification_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create agent with HTTP transport (now uses StreamableHTTPConnectionParams)
agent, mcp_tools = create_clarification_agent(transport="http")
print("✓ Agent created with StreamableHTTPConnectionParams")

# Create runner
session_service = InMemorySessionService()
runner = Runner(
    app_name="mdzen",
    agent=agent,
    session_service=session_service,
)

async def run_clarification():
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state={"session_dir": session_dir},
    )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_request)],
    )
    
    print("🔄 Running agent...")
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state

try:
    final_response, session_state = await run_clarification()
    await close_toolsets(mcp_tools)
    
    if session_state.get("simulation_brief"):
        brief = session_state["simulation_brief"]
        if isinstance(brief, str):
            try:
                brief = json.loads(brief)
            except:
                pass
        
        if isinstance(brief, dict):
            mdzen_state["simulation_brief"] = brief
            
            print("\n✅ SUCCESS! Clarification complete!")
            print("-" * 60)
            print("📋 Generated SimulationBrief:")
            for key, val in brief.items():
                if val is not None:
                    print(f"   • {key}: {val}")
        else:
            # Agent returned clarification questions (not a JSON brief)
            print("\n🤖 Agent needs more information:")
            print(brief[:500] if len(brief) > 500 else brief)
    else:
        print("\n🤖 Agent response:")
        print(final_response[:500] if final_response else "No response")
        
except Exception as e:
    import traceback
    print(f"\n❌ Error: {e}")
    traceback.print_exc()
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

In [12]:
# Check current state
import sys
print(f"Python: {sys.version}")
print(f"IN_COLAB: {'google.colab' in sys.modules}")

# Check if mdzen_state exists
if 'mdzen_state' in dir():
    print(f"\nmdzen_state exists:")
    print(f"  session_dir: {mdzen_state.get('session_dir')}")
    print(f"  simulation_brief: {mdzen_state.get('simulation_brief') is not None}")
else:
    print("\nmdzen_state does NOT exist - need to run Setup and Step 1 first")

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
IN_COLAB: True

mdzen_state exists:
  session_dir: /content/mdzen/outputs/job_f0t81hxs
  simulation_brief: True


In [13]:
# Check simulation brief content
print("Simulation Brief:")
brief = mdzen_state.get('simulation_brief')
if isinstance(brief, dict):
    for k, v in brief.items():
        if v is not None:
            print(f"  {k}: {v}")
else:
    print(f"  Type: {type(brief)}")
    print(f"  Value: {brief}")

Simulation Brief:
  pdb_id: 1AKE
  select_chains: ['A']
  charge_method: bcc
  atom_type: gaff2
  include_types: ['protein', 'water']
  ph: 7.0
  cap_termini: False
  box_padding: 12.0
  cubic_box: True
  salt_concentration: 0.15
  cation_type: Na+
  anion_type: Cl-
  is_membrane: False
  force_field: ff19SB
  water_model: tip3p
  temperature: 300.0
  pressure_bar: 1.0
  timestep: 2.0
  simulation_time_ns: 0.1
  minimize_steps: 500
  nonbonded_cutoff: 10.0
  constraints: HBonds
  output_frequency_ps: 10.0
  use_boltz2_docking: True
  use_msa: True
  num_models: 5
  output_formats: ['amber']


In [14]:
# Run Step 2 - Run Complete Workflow
run_simulation_step = True

import sys
import json
import traceback
from pathlib import Path
import time
import asyncio

brief = mdzen_state["simulation_brief"]
session_dir = Path(mdzen_state["session_dir"])

print("=" * 60)
print(f"  🚀 Starting MD Workflow for {brief.get('pdb_id', 'Unknown')}")
print("  📡 Using ADK Runner + MCP Streamable HTTP Transport")
print("=" * 60)

# Import shared agent (same as main.py!)
from mdzen.agents.setup_agent import create_setup_agent
from mdzen.tools.mcp_setup import close_toolsets

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.agents.readonly_context import ReadonlyContext
from google.genai import types

async def warmup_mcp_tools(mcp_tools, max_retries=3, delay=5):
    """Pre-initialize MCP tool connections to avoid timeouts during run_async."""
    print("\n🔌 Warming up MCP connections...")
    
    for i, toolset in enumerate(mcp_tools):
        server_name = f"server_{i+1}"
        for attempt in range(max_retries):
            try:
                # Force session creation by calling get_tools
                # This triggers the lazy connection
                await asyncio.wait_for(
                    toolset.get_tools(readonly_context=None),
                    timeout=60.0
                )
                print(f"   ✅ {server_name} connected")
                break
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"   ⚠️ {server_name} timeout, retry {attempt + 2}/{max_retries}...")
                    await asyncio.sleep(delay)
                else:
                    raise RuntimeError(f"Failed to connect to {server_name}: {e}")
    
    print("   ✅ All MCP servers ready!")

async def run_setup():
    # Create agent with HTTP transport
    print("\n🔧 Creating setup agent...")
    agent, mcp_tools = create_setup_agent(transport="http")
    
    # Warmup: pre-initialize all MCP connections
    await warmup_mcp_tools(mcp_tools)
    
    # Create runner
    session_service = InMemorySessionService()
    runner = Runner(
        app_name="mdzen",
        agent=agent,
        session_service=session_service,
    )
    
    # Initialize session with simulation brief
    initial_state = {
        "session_dir": str(session_dir),
        "simulation_brief": json.dumps(brief) if isinstance(brief, dict) else brief,
        "completed_steps": json.dumps([]),
        "outputs": json.dumps({}),
    }
    
    session = await session_service.create_session(
        app_name="mdzen",
        user_id="colab_user",
        state=initial_state,
    )
    
    # Build the setup request
    steps_to_run = ["prepare_complex", "solvate", "build_topology"]
    if run_simulation_step:
        steps_to_run.append("run_simulation")
    
    request = f"""Execute the MD setup workflow with the following SimulationBrief:

{json.dumps(brief, indent=2)}

Please run these steps in order: {', '.join(steps_to_run)}

Work in the directory: {session_dir}
"""
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=request)],
    )
    
    print("\n🤖 Setup agent is running...")
    print("-" * 60)
    
    final_response = None
    async for event in runner.run_async(
        user_id="colab_user",
        session_id=session.id,
        new_message=message,
    ):
        # Print progress updates from the agent
        if event.content and event.content.parts:
            text = event.content.parts[0].text if hasattr(event.content.parts[0], 'text') else None
            if text and not event.is_final_response():
                # Print intermediate responses (progress updates)
                if any(kw in text.lower() for kw in ['step', 'complete', 'running', 'preparing', 'building']):
                    print(f"   {text[:200]}...")
            
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    # Get outputs from session state
    updated_session = await session_service.get_session(
        app_name="mdzen",
        user_id="colab_user",
        session_id=session.id,
    )
    
    return final_response, updated_session.state, mcp_tools

try:
    start_time = time.time()
    
    # Colab allows direct await
    final_response, session_state, mcp_tools = await run_setup()
    
    # Close MCP connections
    await close_toolsets(mcp_tools)
    
    elapsed = time.time() - start_time
    
    # Extract outputs from session state
    outputs = session_state.get("outputs", {})
    if isinstance(outputs, str):
        try:
            outputs = json.loads(outputs)
        except:
            outputs = {}
    
    completed = session_state.get("completed_steps", [])
    if isinstance(completed, str):
        try:
            completed = json.loads(completed)
        except:
            completed = []
    
    # Store outputs for visualization
    mdzen_state["workflow_outputs"] = outputs
    
    print()
    print("=" * 60)
    print("  🎉 Workflow Complete!")
    print("=" * 60)
    print(f"  ⏱️ Time: {elapsed/60:.1f} min")
    print(f"  ✅ Steps completed: {', '.join(completed) if completed else 'None'}")
    print(f"  📁 Output: {session_dir}")
    
    # List key output files
    if outputs:
        print()
        print("  📦 Generated files:")
        for key, path in outputs.items():
            if path:
                print(f"     • {key}: {Path(path).name if isinstance(path, str) else path}")
    
    print()
    if final_response:
        print("  📝 Agent summary:")
        # Print first 500 chars of response
        summary = final_response[:500] + "..." if len(final_response) > 500 else final_response
        for line in summary.split('\n'):
            print(f"     {line}")
    
    print()
    print("  👉 Run the next cell to visualize the trajectory")
    
except Exception as e:
    print()
    print("=" * 60)
    print(f"  ❌ Error: {e}")
    print("=" * 60)
    print(traceback.format_exc())
    
    # Cleanup
    try:
        await close_toolsets(mcp_tools)
    except:
        pass

  🚀 Starting MD Workflow for 1AKE
  📡 Using ADK Runner + MCP Streamable HTTP Transport

🔧 Creating setup agent...

🔌 Warming up MCP connections...
   ✅ server_1 connected


CancelledError: Cancelled via cancel scope 7b5b4a489310

In [15]:
# Check which MCP servers are running
import subprocess
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
for line in result.stdout.split('\n'):
    if 'server' in line.lower() and 'python' in line.lower():
        print(line)

root          87  0.5  1.1 403056 149724 ?       Sl   07:47   0:06 /usr/bin/python3 /usr/local/bin/jupyter-server --debug --transport="ipc" --ip=172.28.0.12 --ServerApp.token= --port=9000 --FileContentsManager.root_dir=/ --FileContentsManager.allow_hidden=True --ServerApp.log_format="|%(levelname)s|%(message)s" --ServerApp.iopub_data_rate_limit=1e10 --MappingKernelManager.root_dir=/content
root        2053  3.0  0.9 224260 132240 ?       Sl   07:53   0:24 /usr/bin/python3 /content/mdzen/servers/research_server.py --http --port 8001
root        2058  3.0  0.9 235624 129696 ?       Sl   07:53   0:23 /usr/bin/python3 /content/mdzen/servers/md_simulation_server.py --http --port 8006


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [16]:
# Start all missing MCP servers
import subprocess
import time
import os

os.chdir('/content/mdzen')

servers = [
    ('structure_server.py', 8002),
    ('genesis_server.py', 8003),
    ('solvation_server.py', 8004),
    ('amber_server.py', 8005),
]

for server_file, port in servers:
    server_path = f'/content/mdzen/servers/{server_file}'
    print(f"Starting {server_file} on port {port}...")
    proc = subprocess.Popen(
        ['/usr/bin/python3', server_path, '--http', '--port', str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        start_new_session=True
    )
    print(f"  PID: {proc.pid}")

# Wait for servers to start
print("\nWaiting 10s for servers to initialize...")
time.sleep(10)

# Verify all servers are running
print("\nChecking server processes:")
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
count = 0
for line in result.stdout.split('\n'):
    if '_server.py' in line and 'python' in line.lower():
        # Extract just the server name
        if 'research' in line:
            print("  ✅ research_server (8001)")
        elif 'structure' in line:
            print("  ✅ structure_server (8002)")
        elif 'genesis' in line:
            print("  ✅ genesis_server (8003)")
        elif 'solvation' in line:
            print("  ✅ solvation_server (8004)")
        elif 'amber' in line:
            print("  ✅ amber_server (8005)")
        elif 'md_simulation' in line:
            print("  ✅ md_simulation_server (8006)")
        count += 1

print(f"\nTotal: {count}/6 servers running")

Starting structure_server.py on port 8002...
  PID: 5421
Starting genesis_server.py on port 8003...
  PID: 5422
Starting solvation_server.py on port 8004...
  PID: 5423
Starting amber_server.py on port 8005...
  PID: 5424

Waiting 10s for servers to initialize...

Checking server processes:
  ✅ research_server (8001)
  ✅ md_simulation_server (8006)

Total: 2/6 servers running


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [17]:
# Check why servers are failing - run structure_server directly
import subprocess
result = subprocess.run(
    ['/usr/bin/python3', '/content/mdzen/servers/structure_server.py', '--http', '--port', '8002'],
    capture_output=True,
    text=True,
    timeout=5
)
print("STDOUT:", result.stdout[:1000] if result.stdout else "(empty)")
print("STDERR:", result.stderr[:2000] if result.stderr else "(empty)")
print("Return code:", result.returncode)

STDOUT: (empty)
STDERR: Traceback (most recent call last):
  File "/content/mdzen/servers/structure_server.py", line 30, in <module>
    from pdbfixer import PDBFixer  # noqa: E402
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'pdbfixer'

Return code: 1


In [18]:
# Check all server dependencies
import subprocess

servers_deps = {
    'structure_server.py': ['pdbfixer', 'rdkit', 'openmm'],
    'genesis_server.py': ['boltz'],  
    'solvation_server.py': ['parmed'],
    'amber_server.py': ['parmed'],
}

print("Checking missing dependencies...\n")

missing = []
for server, deps in servers_deps.items():
    print(f"{server}:")
    for dep in deps:
        try:
            __import__(dep)
            print(f"  ✅ {dep}")
        except ImportError:
            print(f"  ❌ {dep} - MISSING")
            missing.append(dep)

print(f"\nMissing: {missing}")

Checking missing dependencies...

structure_server.py:
  ❌ pdbfixer - MISSING
  ✅ rdkit
  ❌ openmm - MISSING
genesis_server.py:
  ❌ boltz - MISSING
solvation_server.py:
  ❌ parmed - MISSING
amber_server.py:
  ❌ parmed - MISSING

Missing: ['pdbfixer', 'openmm', 'boltz', 'parmed', 'parmed']


In [19]:
# Check if conda packages are installed
import subprocess
import os

# Check conda environment
result = subprocess.run(['mamba', 'list'], capture_output=True, text=True)
packages = ['openmm', 'pdbfixer', 'parmed', 'ambertools']
print("Conda packages:")
for pkg in packages:
    if pkg in result.stdout.lower():
        print(f"  ✅ {pkg}")
    else:
        print(f"  ❌ {pkg}")

# Check Python path
import sys
print(f"\nPython: {sys.executable}")
print(f"sys.path[0]: {sys.path[0] if sys.path else 'N/A'}")

Conda packages:
  ❌ openmm
  ❌ pdbfixer
  ❌ parmed
  ❌ ambertools

Python: /usr/bin/python3
sys.path[0]: /content/mdzen


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [20]:
# Install missing scientific packages via mamba
import subprocess
import sys

print("Installing scientific packages via mamba...")
print("This may take a few minutes...\n")

# Install the key packages
result = subprocess.run(
    ['mamba', 'install', '-y', '-c', 'conda-forge', 
     'openmm', 'pdbfixer', 'parmed', 'ambertools'],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Installation successful!")
else:
    print("❌ Installation failed:")
    print(result.stderr[-2000:] if result.stderr else result.stdout[-2000:])

Installing scientific packages via mamba...
This may take a few minutes...



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


✅ Installation successful!


In [21]:
# Check which Python has the packages
import subprocess

# Check conda python
result = subprocess.run(
    ['/usr/local/bin/python', '-c', 'import openmm; print("openmm OK"); import pdbfixer; print("pdbfixer OK"); import parmed; print("parmed OK")'],
    capture_output=True,
    text=True
)
print("Conda Python (/usr/local/bin/python):")
print(result.stdout if result.stdout else result.stderr)

# Check system python  
result2 = subprocess.run(
    ['/usr/bin/python3', '-c', 'import openmm; print("openmm OK")'],
    capture_output=True,
    text=True
)
print("\nSystem Python (/usr/bin/python3):")
print(result2.stdout if result2.stdout else result2.stderr[:500])

Conda Python (/usr/local/bin/python):
openmm OK
pdbfixer OK
parmed OK


System Python (/usr/bin/python3):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'openmm'



In [22]:
# Kill existing servers and restart with conda Python
import subprocess
import time
import os

# Kill existing server processes
print("Killing existing MCP servers...")
result = subprocess.run(['pkill', '-f', '_server.py'], capture_output=True)
time.sleep(2)

# Verify they're dead
result = subprocess.run(['pgrep', '-f', '_server.py'], capture_output=True, text=True)
if result.stdout.strip():
    print("  Some servers still running, forcing kill...")
    subprocess.run(['pkill', '-9', '-f', '_server.py'])
    time.sleep(1)

print("  ✅ All servers stopped\n")

# Start all servers with conda Python
os.chdir('/content/mdzen')
PYTHON = '/usr/local/bin/python'

servers = [
    ('research_server.py', 8001),
    ('structure_server.py', 8002),
    ('genesis_server.py', 8003),
    ('solvation_server.py', 8004),
    ('amber_server.py', 8005),
    ('md_simulation_server.py', 8006),
]

print(f"Starting all MCP servers with {PYTHON}...")
for server_file, port in servers:
    server_path = f'/content/mdzen/servers/{server_file}'
    proc = subprocess.Popen(
        [PYTHON, server_path, '--http', '--port', str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        start_new_session=True
    )
    print(f"  {server_file} (port {port}) - PID {proc.pid}")

print("\nWaiting 15s for servers to initialize...")
time.sleep(15)

# Verify all servers are running
print("\nVerifying servers:")
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
running = []
for line in result.stdout.split('\n'):
    if '_server.py' in line and 'python' in line.lower():
        for name in ['research', 'structure', 'genesis', 'solvation', 'amber', 'md_simulation']:
            if name in line:
                running.append(name)
                
expected = ['research', 'structure', 'genesis', 'solvation', 'amber', 'md_simulation']
for name in expected:
    if name in running:
        print(f"  ✅ {name}_server")
    else:
        print(f"  ❌ {name}_server - NOT RUNNING")

print(f"\nTotal: {len(running)}/6 servers running")

Killing existing MCP servers...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


  ✅ All servers stopped

Starting all MCP servers with /usr/local/bin/python...
  research_server.py (port 8001) - PID 6103
  structure_server.py (port 8002) - PID 6104
  genesis_server.py (port 8003) - PID 6105
  solvation_server.py (port 8004) - PID 6106
  amber_server.py (port 8005) - PID 6107
  md_simulation_server.py (port 8006) - PID 6108

Waiting 15s for servers to initialize...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



Verifying servers:
  ❌ research_server - NOT RUNNING
  ❌ structure_server - NOT RUNNING
  ❌ genesis_server - NOT RUNNING
  ❌ solvation_server - NOT RUNNING
  ❌ amber_server - NOT RUNNING
  ❌ md_simulation_server - NOT RUNNING

Total: 0/6 servers running


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [23]:
# Debug: run one server and capture error
import subprocess

result = subprocess.run(
    ['/usr/local/bin/python', '/content/mdzen/servers/research_server.py', '--http', '--port', '8001'],
    capture_output=True,
    text=True,
    timeout=5,
    cwd='/content/mdzen'
)
print("STDOUT:", result.stdout[:500] if result.stdout else "(empty)")
print("\nSTDERR:", result.stderr[:2000] if result.stderr else "(empty)")
print("\nReturn code:", result.returncode)

STDOUT: (empty)

STDERR: Traceback (most recent call last):
  File "/content/mdzen/servers/research_server.py", line 19, in <module>
    import httpx
ModuleNotFoundError: No module named 'httpx'


Return code: 1


In [24]:
# Install missing pip packages in conda Python
import subprocess

print("Installing missing pip packages in conda Python...")

packages = ['httpx', 'fastmcp', 'pydantic', 'google-adk', 'litellm', 'anthropic']

result = subprocess.run(
    ['/usr/local/bin/pip', 'install'] + packages,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Installation successful!")
else:
    print("❌ Error:")
    print(result.stderr[-1500:])

Installing missing pip packages in conda Python...
❌ Error:
error: resolution-too-deep

× Dependency resolution exceeded maximum depth
╰─> Pip cannot resolve the current dependencies as the dependency graph is too complex for pip to solve efficiently.

hint: Try adding lower bounds to constrain your dependencies, for example: 'package>=2.0.0' instead of just 'package'.

Link: https://pip.pypa.io/en/stable/topics/dependency-resolution/#handling-resolution-too-deep-errors



In [25]:
# Install directly to system Python (faster approach)
import subprocess
import sys

print("Installing to system Python directly...")

# Install openmm and pdbfixer via pip (they're available on PyPI now)
result = subprocess.run(
    [sys.executable, '-m', 'pip', 'install', '--quiet', 'openmm', 'pdbfixer', 'parmed'],
    capture_output=True,
    text=True,
    timeout=120
)

if result.returncode == 0:
    print("✅ Done!")
else:
    print(f"Status: {result.returncode}")
    print(result.stderr[-500:] if result.stderr else "")

Installing to system Python directly...
Status: 1
ERROR: Could not find a version that satisfies the requirement pdbfixer (from versions: none)
ERROR: No matching distribution found for pdbfixer



In [26]:
# Try installing pip packages one by one to conda Python
import subprocess

packages = ['httpx', 'fastmcp', 'mcp']

for pkg in packages:
    print(f"Installing {pkg}...", end=" ")
    result = subprocess.run(
        ['/usr/local/bin/pip', 'install', '--quiet', '--no-deps', pkg],
        capture_output=True, text=True, timeout=60
    )
    if result.returncode == 0:
        print("✅")
    else:
        print(f"❌ {result.stderr[:100]}")

Installing httpx... ✅
Installing fastmcp... ✅
Installing mcp... ✅


In [27]:
# Quick test - run research_server
import subprocess

result = subprocess.run(
    ['/usr/local/bin/python', '/content/mdzen/servers/research_server.py', '--http', '--port', '8001'],
    capture_output=True, text=True, timeout=5, cwd='/content/mdzen'
)
print("STDERR:", result.stderr[:1500] if result.stderr else "(none)")

STDERR: Traceback (most recent call last):
  File "/content/mdzen/servers/research_server.py", line 20, in <module>
    from fastmcp import FastMCP
  File "/usr/local/lib/python3.12/site-packages/fastmcp/__init__.py", line 5, in <module>
    from fastmcp.settings import Settings
  File "/usr/local/lib/python3.12/site-packages/fastmcp/settings.py", line 11, in <module>
    from pydantic import Field, ImportString, field_validator
ModuleNotFoundError: No module named 'pydantic'



In [28]:
# Install more dependencies quickly
import subprocess

packages = ['pydantic', 'pydantic-settings', 'anyio', 'starlette', 'uvicorn', 'sse-starlette', 'httpx-sse']

for pkg in packages:
    print(f"{pkg}...", end=" ")
    result = subprocess.run(
        ['/usr/local/bin/pip', 'install', '--quiet', pkg],
        capture_output=True, text=True, timeout=60
    )
    print("✅" if result.returncode == 0 else "❌")

pydantic... ✅
pydantic-settings... ✅
anyio... ✅
starlette... ✅
uvicorn... ✅
sse-starlette... ✅
httpx-sse... ✅


In [29]:
# Test research_server again
import subprocess

result = subprocess.run(
    ['/usr/local/bin/python', '/content/mdzen/servers/research_server.py', '--http', '--port', '8001'],
    capture_output=True, text=True, timeout=5, cwd='/content/mdzen'
)
if result.stderr:
    # Check if it's just a timeout (server started successfully)
    if 'Traceback' in result.stderr:
        print("Error:", result.stderr[:1000])
    else:
        print("Server output:", result.stderr[:500])

Error: Traceback (most recent call last):
  File "/content/mdzen/servers/research_server.py", line 20, in <module>
    from fastmcp import FastMCP
  File "/usr/local/lib/python3.12/site-packages/fastmcp/__init__.py", line 5, in <module>
    from fastmcp.settings import Settings
  File "/usr/local/lib/python3.12/site-packages/fastmcp/settings.py", line 17, in <module>
    from fastmcp.utilities.logging import get_logger
  File "/usr/local/lib/python3.12/site-packages/fastmcp/utilities/logging.py", line 7, in <module>
    from rich.console import Console
ModuleNotFoundError: No module named 'rich'



In [30]:
# Install remaining fastmcp dependencies
import subprocess

packages = ['rich', 'typer', 'python-dotenv', 'exceptiongroup', 'opentelemetry-api', 'opentelemetry-sdk']

result = subprocess.run(
    ['/usr/local/bin/pip', 'install', '--quiet'] + packages,
    capture_output=True, text=True, timeout=120
)
print("✅ Done" if result.returncode == 0 else f"❌ {result.stderr[:300]}")

# Also install mdzen package
result2 = subprocess.run(
    ['/usr/local/bin/pip', 'install', '-e', '/content/mdzen'],
    capture_output=True, text=True, timeout=60
)
print("mdzen:", "✅" if result2.returncode == 0 else "❌")

✅ Done


TimeoutExpired: Command '['/usr/local/bin/pip', 'install', '-e', '/content/mdzen']' timed out after 60 seconds

In [31]:
# Test server directly now
import subprocess

result = subprocess.run(
    ['/usr/local/bin/python', '/content/mdzen/servers/research_server.py', '--http', '--port', '8001'],
    capture_output=True, text=True, timeout=5, cwd='/content/mdzen'
)
print(result.stderr[:1500] if result.stderr else "No error - server likely started!")

Traceback (most recent call last):
  File "/content/mdzen/servers/research_server.py", line 20, in <module>
    from fastmcp import FastMCP
  File "/usr/local/lib/python3.12/site-packages/fastmcp/__init__.py", line 10, in <module>
    _configure_logging(
  File "/usr/local/lib/python3.12/site-packages/fastmcp/utilities/logging.py", line 74, in configure_logging
    import mcp
  File "/usr/local/lib/python3.12/site-packages/mcp/__init__.py", line 4, in <module>
    from .server.session import ServerSession
  File "/usr/local/lib/python3.12/site-packages/mcp/server/__init__.py", line 1, in <module>
    from .fastmcp import FastMCP
  File "/usr/local/lib/python3.12/site-packages/mcp/server/fastmcp/__init__.py", line 7, in <module>
    from .server import Context, FastMCP
  File "/usr/local/lib/python3.12/site-packages/mcp/server/fastmcp/server.py", line 57, in <module>
    from mcp.server.lowlevel.helper_types import ReadResourceContents
  File "/usr/local/lib/python3.12/site-packages/mcp

In [32]:
# Install mcp with all dependencies properly
import subprocess

print("Installing mcp with full dependencies...")
result = subprocess.run(
    ['/usr/local/bin/pip', 'install', '--quiet', 'mcp[cli]', 'fastmcp[all]'],
    capture_output=True, text=True, timeout=180
)
print("✅ Done" if result.returncode == 0 else f"❌ {result.stderr[-500:]}")

Installing mcp with full dependencies...
✅ Done


In [33]:
# Test server now
import subprocess
import time

# Start all servers
servers = [
    ('research_server.py', 8001),
    ('structure_server.py', 8002),
    ('solvation_server.py', 8004),
    ('amber_server.py', 8005),
    ('md_simulation_server.py', 8006),
]

print("Starting servers...")
for server, port in servers:
    proc = subprocess.Popen(
        ['/usr/local/bin/python', f'/content/mdzen/servers/{server}', '--http', '--port', str(port)],
        stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, start_new_session=True
    )
    print(f"  {server}: PID {proc.pid}")

time.sleep(8)

# Check which are running
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
count = sum(1 for line in result.stdout.split('\n') if '_server.py' in line and 'python' in line)
print(f"\n{count}/5 servers running")

Starting servers...
  research_server.py: PID 23951
  structure_server.py: PID 23952
  solvation_server.py: PID 23953
  amber_server.py: PID 23954
  md_simulation_server.py: PID 23955

5/5 servers running


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [34]:
# Run Step 2 - Run Complete Workflow
run_simulation_step = True

import sys
import json
import traceback
from pathlib import Path
import time
import asyncio

brief = mdzen_state["simulation_brief"]
session_dir = Path(mdzen_state["session_dir"])

print("=" * 60)
print(f"  🚀 Starting MD Workflow for {brief.get('pdb_id', 'Unknown')}")
print("=" * 60)

from mdzen.agents.setup_agent import create_setup_agent
from mdzen.tools.mcp_setup import close_toolsets
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

async def warmup_mcp_tools(mcp_tools, max_retries=3, delay=5):
    print("\n🔌 Warming up MCP connections...")
    for i, toolset in enumerate(mcp_tools):
        for attempt in range(max_retries):
            try:
                await asyncio.wait_for(toolset.get_tools(readonly_context=None), timeout=60.0)
                print(f"   ✅ server_{i+1}")
                break
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"   ⚠️ server_{i+1} retry {attempt + 2}...")
                    await asyncio.sleep(delay)
                else:
                    raise RuntimeError(f"Failed: {e}")
    print("   ✅ All ready!")

async def run_setup():
    print("\n🔧 Creating setup agent...")
    agent, mcp_tools = create_setup_agent(transport="http")
    await warmup_mcp_tools(mcp_tools)
    
    session_service = InMemorySessionService()
    runner = Runner(app_name="mdzen", agent=agent, session_service=session_service)
    
    initial_state = {
        "session_dir": str(session_dir),
        "simulation_brief": json.dumps(brief),
        "completed_steps": json.dumps([]),
        "outputs": json.dumps({}),
    }
    session = await session_service.create_session(app_name="mdzen", user_id="colab_user", state=initial_state)
    
    steps = ["prepare_complex", "solvate", "build_topology"]
    if run_simulation_step:
        steps.append("run_simulation")
    
    request = f"Execute MD workflow: {json.dumps(brief, indent=2)}\nSteps: {', '.join(steps)}\nDir: {session_dir}"
    message = types.Content(role="user", parts=[types.Part(text=request)])
    
    print("\n🤖 Running...")
    print("-" * 60)
    
    final_response = None
    async for event in runner.run_async(user_id="colab_user", session_id=session.id, new_message=message):
        if event.is_final_response() and event.content:
            final_response = event.content.parts[0].text if event.content.parts else None
    
    updated = await session_service.get_session(app_name="mdzen", user_id="colab_user", session_id=session.id)
    return final_response, updated.state, mcp_tools

try:
    start = time.time()
    final_response, session_state, mcp_tools = await run_setup()
    await close_toolsets(mcp_tools)
    
    outputs = json.loads(session_state.get("outputs", "{}")) if isinstance(session_state.get("outputs"), str) else session_state.get("outputs", {})
    completed = json.loads(session_state.get("completed_steps", "[]")) if isinstance(session_state.get("completed_steps"), str) else session_state.get("completed_steps", [])
    mdzen_state["workflow_outputs"] = outputs
    
    print(f"\n{'='*60}\n  🎉 Complete! ({(time.time()-start)/60:.1f} min)")
    print(f"  Steps: {completed}\n  Files: {list(outputs.keys())}")
except Exception as e:
    print(f"\n❌ Error: {e}")
    traceback.print_exc()

  🚀 Starting MD Workflow for 1AKE

🔧 Creating setup agent...

🔌 Warming up MCP connections...


/usr/lib/python3.12/contextlib.py:105: DeprecationWarning: Use `streamable_http_client` instead.
  self.gen = func(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_toolset.py:181: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


   ✅ server_1
   ✅ server_2


CancelledError: Cancelled via cancel scope 7b5b4a563dd0

In [35]:
# Check which ports are listening
import subprocess

result = subprocess.run(['ss', '-tlnp'], capture_output=True, text=True)
print("Listening ports:")
for line in result.stdout.split('\n'):
    if '800' in line:
        print(line)

Listening ports:
LISTEN 0      2048         0.0.0.0:8006       0.0.0.0:*    users:(("python",pid=23955,fd=6))      
LISTEN 0      2048         0.0.0.0:8001       0.0.0.0:*    users:(("python",pid=23951,fd=6))      
LISTEN 0      2048         0.0.0.0:8002       0.0.0.0:*    users:(("python",pid=23952,fd=6))      


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [36]:
# Check why solvation and amber servers crashed
import subprocess

# Test solvation server
print("=== solvation_server ===")
result = subprocess.run(
    ['/usr/local/bin/python', '/content/mdzen/servers/solvation_server.py', '--http', '--port', '8004'],
    capture_output=True, text=True, timeout=5, cwd='/content/mdzen'
)
print(result.stderr[:1000] if result.stderr else "OK")

print("\n=== amber_server ===")
result2 = subprocess.run(
    ['/usr/local/bin/python', '/content/mdzen/servers/amber_server.py', '--http', '--port', '8005'],
    capture_output=True, text=True, timeout=5, cwd='/content/mdzen'
)
print(result2.stderr[:1000] if result2.stderr else "OK")

=== solvation_server ===
Traceback (most recent call last):
  File "/content/mdzen/servers/solvation_server.py", line 31, in <module>
    from mdzen.config import get_timeout  # noqa: E402
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'mdzen'


=== amber_server ===
Traceback (most recent call last):
  File "/content/mdzen/servers/amber_server.py", line 30, in <module>
    from mdzen.config import get_timeout  # noqa: E402
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'mdzen'



In [37]:
# Install mdzen package in conda Python (quick way - just add to path)
import subprocess

# Add mdzen to PYTHONPATH instead of installing
import os
os.environ['PYTHONPATH'] = '/content/mdzen/src:' + os.environ.get('PYTHONPATH', '')

# Start solvation and amber with PYTHONPATH set
print("Starting missing servers...")

env = os.environ.copy()
env['PYTHONPATH'] = '/content/mdzen/src'

for server, port in [('solvation_server.py', 8004), ('amber_server.py', 8005)]:
    proc = subprocess.Popen(
        ['/usr/local/bin/python', f'/content/mdzen/servers/{server}', '--http', '--port', str(port)],
        stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, start_new_session=True, env=env
    )
    print(f"  {server}: PID {proc.pid}")

import time
time.sleep(5)

# Check ports
result = subprocess.run(['ss', '-tlnp'], capture_output=True, text=True)
print("\nListening ports:")
for line in result.stdout.split('\n'):
    if '800' in line:
        print(line)

Starting missing servers...
  solvation_server.py: PID 24356
  amber_server.py: PID 24357

Listening ports:
LISTEN 0      2048         0.0.0.0:8005       0.0.0.0:*    users:(("python",pid=24357,fd=6))      
LISTEN 0      2048         0.0.0.0:8004       0.0.0.0:*    users:(("python",pid=24356,fd=6))      
LISTEN 0      2048         0.0.0.0:8006       0.0.0.0:*    users:(("python",pid=23955,fd=6))      
LISTEN 0      2048         0.0.0.0:8001       0.0.0.0:*    users:(("python",pid=23951,fd=6))      
LISTEN 0      2048         0.0.0.0:8002       0.0.0.0:*    users:(("python",pid=23952,fd=6))      


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [38]:
# Run Step 2 again
run_simulation_step = True

import sys, json, traceback, time, asyncio
from pathlib import Path

brief = mdzen_state["simulation_brief"]
session_dir = Path(mdzen_state["session_dir"])

print(f"🚀 Starting MD Workflow for {brief.get('pdb_id')}")

from mdzen.agents.setup_agent import create_setup_agent
from mdzen.tools.mcp_setup import close_toolsets
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

async def warmup(mcp_tools):
    print("🔌 Connecting...")
    for i, t in enumerate(mcp_tools):
        for attempt in range(3):
            try:
                await asyncio.wait_for(t.get_tools(readonly_context=None), timeout=90.0)
                print(f"   ✅ server_{i+1}")
                break
            except Exception as e:
                if attempt < 2:
                    print(f"   ⚠️ server_{i+1} retry...")
                    await asyncio.sleep(5)
                else:
                    raise

async def run():
    agent, mcp_tools = create_setup_agent(transport="http")
    await warmup(mcp_tools)
    
    svc = InMemorySessionService()
    runner = Runner(app_name="mdzen", agent=agent, session_service=svc)
    
    session = await svc.create_session(
        app_name="mdzen", user_id="u", 
        state={"session_dir": str(session_dir), "simulation_brief": json.dumps(brief), 
               "completed_steps": "[]", "outputs": "{}"}
    )
    
    steps = ["prepare_complex", "solvate", "build_topology"] + (["run_simulation"] if run_simulation_step else [])
    msg = types.Content(role="user", parts=[types.Part(text=f"Run: {steps}\nBrief: {json.dumps(brief)}\nDir: {session_dir}")])
    
    print("🤖 Agent running...")
    resp = None
    async for e in runner.run_async(user_id="u", session_id=session.id, new_message=msg):
        if e.is_final_response() and e.content:
            resp = e.content.parts[0].text if e.content.parts else None
    
    s = await svc.get_session(app_name="mdzen", user_id="u", session_id=session.id)
    return resp, s.state, mcp_tools

try:
    t0 = time.time()
    resp, state, tools = await run()
    await close_toolsets(tools)
    
    out = json.loads(state.get("outputs","{}")) if isinstance(state.get("outputs"),str) else state.get("outputs",{})
    done = json.loads(state.get("completed_steps","[]")) if isinstance(state.get("completed_steps"),str) else state.get("completed_steps",[])
    mdzen_state["workflow_outputs"] = out
    
    print(f"\n🎉 Done in {(time.time()-t0)/60:.1f} min")
    print(f"Steps: {done}")
    print(f"Files: {list(out.keys())}")
except Exception as e:
    print(f"\n❌ {e}")
    traceback.print_exc()

🚀 Starting MD Workflow for 1AKE
🔌 Connecting...
   ✅ server_1


/usr/lib/python3.12/contextlib.py:105: DeprecationWarning: Use `streamable_http_client` instead.
  self.gen = func(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_toolset.py:181: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


   ✅ server_2


CancelledError: Cancelled via cancel scope 7b5b33f8d940

In [39]:
# Check what toolsets the setup agent creates
from mdzen.agents.setup_agent import create_setup_agent

agent, mcp_tools = create_setup_agent(transport="http")

print(f"Number of MCP toolsets: {len(mcp_tools)}")
for i, t in enumerate(mcp_tools):
    # Get the URL from the connection params
    params = t._connection_params
    print(f"  {i+1}: {params}")

Number of MCP toolsets: 6
  1: url='http://localhost:8001/mcp' headers=None timeout=120.0 sse_read_timeout=300.0 terminate_on_close=True
  2: url='http://localhost:8002/mcp' headers=None timeout=120.0 sse_read_timeout=300.0 terminate_on_close=True
  3: url='http://localhost:8003/mcp' headers=None timeout=120.0 sse_read_timeout=300.0 terminate_on_close=True
  4: url='http://localhost:8004/mcp' headers=None timeout=120.0 sse_read_timeout=300.0 terminate_on_close=True
  5: url='http://localhost:8005/mcp' headers=None timeout=120.0 sse_read_timeout=300.0 terminate_on_close=True
  6: url='http://localhost:8006/mcp' headers=None timeout=120.0 sse_read_timeout=300.0 terminate_on_close=True


In [40]:
# Start genesis_server
import subprocess, os, time

env = os.environ.copy()
env['PYTHONPATH'] = '/content/mdzen/src'

proc = subprocess.Popen(
    ['/usr/local/bin/python', '/content/mdzen/servers/genesis_server.py', '--http', '--port', '8003'],
    stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, start_new_session=True, env=env
)
print(f"genesis_server: PID {proc.pid}")
time.sleep(5)

# Check all ports
result = subprocess.run(['ss', '-tlnp'], capture_output=True, text=True)
ports = []
for line in result.stdout.split('\n'):
    if '800' in line:
        for p in ['8001','8002','8003','8004','8005','8006']:
            if f':{p}' in line:
                ports.append(p)
print(f"Ports listening: {sorted(ports)}")

genesis_server: PID 24601
Ports listening: ['8001', '8002', '8003', '8004', '8005', '8006']


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Run Step 2 - final attempt
import sys, json, traceback, time, asyncio
from pathlib import Path

brief = mdzen_state["simulation_brief"]
session_dir = Path(mdzen_state["session_dir"])

print(f"🚀 MD Workflow for {brief.get('pdb_id')}")

from mdzen.agents.setup_agent import create_setup_agent
from mdzen.tools.mcp_setup import close_toolsets
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

async def warmup(tools):
    print("🔌 Connecting to 6 servers...")
    for i, t in enumerate(tools):
        for attempt in range(3):
            try:
                await asyncio.wait_for(t.get_tools(readonly_context=None), timeout=120.0)
                print(f"   ✅ {i+1}/6")
                break
            except Exception as e:
                if attempt < 2:
                    print(f"   ⚠️ {i+1} retry {attempt+2}...")
                    await asyncio.sleep(8)
                else:
                    raise RuntimeError(f"Server {i+1} failed: {e}")
    print("   All connected!")

async def run():
    agent, tools = create_setup_agent(transport="http")
    await warmup(tools)
    
    svc = InMemorySessionService()
    runner = Runner(app_name="mdzen", agent=agent, session_service=svc)
    
    session = await svc.create_session(
        app_name="mdzen", user_id="u", 
        state={"session_dir": str(session_dir), "simulation_brief": json.dumps(brief), 
               "completed_steps": "[]", "outputs": "{}"}
    )
    
    msg = types.Content(role="user", parts=[types.Part(text=f"Run MD workflow: prepare_complex, solvate, build_topology, run_simulation\nBrief: {json.dumps(brief)}\nDir: {session_dir}")])
    
    print("🤖 Agent running (this takes several minutes)...")
    resp = None
    async for e in runner.run_async(user_id="u", session_id=session.id, new_message=msg):
        if e.is_final_response() and e.content:
            resp = e.content.parts[0].text if e.content.parts else None
    
    s = await svc.get_session(app_name="mdzen", user_id="u", session_id=session.id)
    return resp, s.state, tools

try:
    t0 = time.time()
    resp, state, tools = await run()
    await close_toolsets(tools)
    
    out = json.loads(state.get("outputs","{}")) if isinstance(state.get("outputs"),str) else state.get("outputs",{})
    done = json.loads(state.get("completed_steps","[]")) if isinstance(state.get("completed_steps"),str) else state.get("completed_steps",[])
    mdzen_state["workflow_outputs"] = out
    
    print(f"\n🎉 Done in {(time.time()-t0)/60:.1f} min")
    print(f"Steps: {done}")
    print(f"Files: {list(out.keys())}")
    if resp:
        print(f"\nSummary: {resp[:300]}...")
except Exception as e:
    print(f"\n❌ {e}")
    traceback.print_exc()

🚀 MD Workflow for 1AKE
🔌 Connecting to 6 servers...


/usr/lib/python3.12/contextlib.py:105: DeprecationWarning: Use `streamable_http_client` instead.
  self.gen = func(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_toolset.py:181: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


   ✅ 1/6
   ✅ 2/6
   ✅ 3/6
   ✅ 4/6
   ✅ 5/6
   ✅ 6/6
   All connected!
🤖 Agent running (this takes several minutes)...


/usr/local/lib/python3.12/dist-packages/google/adk/runners.py:1414: DeprecationWarning: deprecated
  save_input_blobs_as_artifacts=run_config.save_input_blobs_as_artifacts,
/usr/local/lib/python3.12/dist-packages/aiohttp/connector.py:963: DeprecationWarning: enable_cleanup_closed ignored because https://github.com/python/cpython/pull/118960 is fixed in Python version sys.version_info(major=3, minor=12, micro=12, releaselevel='final', serial=0)
  super().__init__(


: 

In [ ]:
# Alternative approach: Direct HTTP calls to MCP servers (bypass ADK)
import httpx
import json
import asyncio

async def call_mcp_tool(port: int, tool_name: str, arguments: dict, timeout: float = 300.0):
    """Call MCP tool directly via HTTP POST."""
    url = f"http://localhost:{port}/mcp"
    
    # MCP JSON-RPC format
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": arguments
        }
    }
    
    async with httpx.AsyncClient(timeout=timeout) as client:
        # First initialize session
        init_payload = {
            "jsonrpc": "2.0",
            "id": 0,
            "method": "initialize",
            "params": {
                "protocolVersion": "2024-11-05",
                "capabilities": {},
                "clientInfo": {"name": "direct-client", "version": "1.0"}
            }
        }
        
        resp = await client.post(url, json=init_payload)
        if resp.status_code != 200:
            return {"error": f"Init failed: {resp.status_code}"}
        
        # Call tool
        resp = await client.post(url, json=payload)
        return resp.json()

# Test connection to research server
print("Testing direct MCP call...")
result = await call_mcp_tool(8001, "get_protein_info", {"uniprot_id": "P69441"})
print(f"Result: {json.dumps(result, indent=2)[:500]}")